# (9-6-2021)->optimized-> 11-14-2021
## Optimized Setup: Setting Up Current Maps with Sorted Stable Event Lists.  Whose purpose is to collect CURL and MEC data that corresponds with the Event Lists
### Sequel to `Event Searching pt2`
#### Hector Salinas
---

#### Eric Mentions:
* (9-13-2021)As confirmed by Eric, the components of the curl parameters refer to the coordinate system inputted inot the fgm function( >>fgm(....var_format = * gsm *).  'The output coordinate system should be the same as the input coordinate system (so if you provide the FGM and position data in GSE coordinates, the output vectors will be in GSE coordinates as well).'
* (6-24-21; old email) Yeah, if you return the individual elements of the tuple (instead of the tuple itself), it’ll be:
    * `>>> pos_times, pos = get_data('mms1_mec_r_gsm’)`
    * `>>> radius of position vector: r = np.sqrt(pos[:, 0]**2+pos[:, 1]**2+pos[:, 2]**2)`
    * `pos[:, 0] is the x-component`
    * `pos[:, 1] is the y-component`
    * `pos[:, 2] is the z-component`
***
#### Rick Pointers:
* Burst data is 128 samples/sec and selected for very specific intervals
* Survey data is 16 samples/sec and all through the ROI(region of interest)
    * Burst should only be for studying detailed physics of specific events
    * For the end-goal Current Maps, survey is all we need. But if there's some interesting periods for follow-up studies, we'll start digging into burst data
***
### Script-Goals
* This script will be used to export dataframes that contain data on the CURL params and correpsonding MEC position necessary for creating Current Maps for the bowshowck-magpause region 
1. Load up Curlometer data(GSM coord) for corresponding date 
2. Take the median, mean, and std over the time interval for the desired CURL parameters:
    * curlB components and divB in current units
3. (9-18-2021)Collect median, mean, and std values for corresponding time frame as datapoints for dataframe that will be used elsewhere


***
#### pySPEDAS References
* Jupyter Notebook Viewer: pySPEDAS routine how-tos:
    * https://nbviewer.jupyter.org/github/spedas/mms-examples/blob/master/basic/Fast%20Plasma%20Investigation%20%28FPI%29.ipynb
* Github: pySPEDAS tutorial on many features:
    * https://github.com/spedas/pyspedas_examples/blob/master/pyspedas_examples/tutorials/pySPEDAS%201.0%20tutorial%20-%20dry%20run%20(1%20May%202020).ipynb
    * As indicated by the github tutorial, many tplot variables can be displayed on a single plane using the join_vec() function

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
from scipy import optimize 
from scipy.signal import find_peaks, find_peaks_cwt #displaying peaks on data
import numpy as np
import time
import pandas as pd
import copy
import pytplot

from pyspedas.mms import fgm, fpi, curlometer, mec #import instrument load routines
from pyspedas import tinterpol    #to match MEC data to OMNI 5min time stamps

import pyspedas
from pytplot import tplot, tlimit # plot said tplot variables and specify xrange
from pytplot import tplot_names   # list tplot variables loaded
from pytplot import get_data      # extract data values from tplot variables
from pyspedas import time_string  # convert unix time to string(returns it as 'list object' not numpy array)
from pyspedas import time_double  #convert string back to unix time
from pytplot import store_data    # creating tplot variables
from pytplot import options       # extra tplot features for individial tplots
from pytplot import tplot_options # global tplot options for created tplots
from pytplot import join_vec      # combine multiple tplot variables into a single one



# Section 1:
## Importing Day and Nightside Stable Event Lists(Yr1 to 5) for Certain IMF Clock Angles and extracting last 10min from each 30minute interval
---
#### IMF Orientations to Load/Extract Data from:
* Southward $B_z$: $\theta_{avg,IMF} \geq 135^o$ and $< 225^o$
* Positive $B_y$-led: $\theta_{avg,IMF} \geq 45^o$ and $< 135^o$
* Negative $B_y$-led: $\theta_{avg,IMF} \geq 225^o$ and $< 315^o$
* Northward $B_z$: $\theta_{avg,IMF} \geq 315^o$ and $< 45^o$

    * reference to IMF ClockAng Diagram: https://www.sws.bom.gov.au/Category/Solar/Solar%20Conditions/Solar%20Wind%20Clock%20Angle/Solar%20Wind%20Clock%20Angle.php



#### Importing Dayside(Sept to May) Yrs1-5 Stable IMF Lists

In [2]:
# Importing yearly MEC positional data
    #Extracting MEC radial position(km) and time(unix time) arrays
#Load Dayside MEC Data
yr1_MECposdata = np.load(r'/Users/salinasha/Desktop/MacBoo Codes/Space Physics Codes/Under pySPEDAS_venv/MEC exported Dayside posdata/yr1_MECgsm_posdata.npz')
yr2_MECposdata = np.load(r'/Users/salinasha/Desktop/MacBoo Codes/Space Physics Codes/Under pySPEDAS_venv/MEC exported Dayside posdata/yr2_MECgsm_posdata.npz')
yr3_MECposdata = np.load(r'/Users/salinasha/Desktop/MacBoo Codes/Space Physics Codes/Under pySPEDAS_venv/MEC exported Dayside posdata/yr3_MECgsm_posdata.npz')
yr4_MECposdata = np.load(r'/Users/salinasha/Desktop/MacBoo Codes/Space Physics Codes/Under pySPEDAS_venv/MEC exported Dayside posdata/yr4_MECgsm_posdata.npz')
yr5_MECposdata = np.load(r'/Users/salinasha/Desktop/MacBoo Codes/Space Physics Codes/Under pySPEDAS_venv/MEC exported Dayside posdata/yr5_MECgsm_posdata.npz')
yr6_MECposdata = np.load(r'/Users/salinasha/Desktop/MacBoo Codes/Space Physics Codes/Under pySPEDAS_venv/MEC exported Dayside posdata/Dayside_yr6_MECgsm_posdata.npz')

print(sorted(yr5_MECposdata))

def show1to5DF(df_1, df_2, df_3, df_4, df_5):
    """Displays Yrs 1to5 DFs"""
    display(df_1)
    display(df_2)
    display(df_3)
    display(df_4)
    display(df_5)
    return;

def show1to6DF(df_1, df_2, df_3, df_4, df_5, df_6):
    """Displays Yrs 1to6 DFs"""
    display(df_1)
    display(df_2)
    display(df_3)
    display(df_4)
    display(df_5)
    display(df_6)
    return;

#'''
#Importing Yr 1-5 SoBZ Stable List
df_DaysoBz_yr1 = pd.read_csv(r'/Users/salinasha/Desktop/MacBoo Codes/Space Physics Codes/Under pySPEDAS_venv/'
'Stable Day List_with Bcomps/pdFile_Day_soBz_withBcomp/pdFile_Day_Yr1_7to24Re_30mFilt_Stable_soBz.txt', sep = '\t')
df_DaysoBz_yr2 = pd.read_csv(r'/Users/salinasha/Desktop/MacBoo Codes/Space Physics Codes/Under pySPEDAS_venv/'
'Stable Day List_with Bcomps/pdFile_Day_soBz_withBcomp/pdFile_Day_Yr2_7to24Re_30mFilt_Stable_soBz.txt', sep = '\t')
df_DaysoBz_yr3 = pd.read_csv(r'/Users/salinasha/Desktop/MacBoo Codes/Space Physics Codes/Under pySPEDAS_venv/'
'Stable Day List_with Bcomps/pdFile_Day_soBz_withBcomp/pdFile_Day_Yr3_7to24Re_30mFilt_Stable_soBz.txt', sep = '\t')
df_DaysoBz_yr4 = pd.read_csv(r'/Users/salinasha/Desktop/MacBoo Codes/Space Physics Codes/Under pySPEDAS_venv/'
'Stable Day List_with Bcomps/pdFile_Day_soBz_withBcomp/pdFile_Day_Yr4_7to24Re_30mFilt_Stable_soBz.txt', sep = '\t')
df_DaysoBz_yr5 = pd.read_csv(r'/Users/salinasha/Desktop/MacBoo Codes/Space Physics Codes/Under pySPEDAS_venv/'
'Stable Day List_with Bcomps/pdFile_Day_soBz_withBcomp/pdFile_Day_Yr5_7to24Re_30mFilt_Stable_soBz_noMECgap.txt', sep = '\t')
df_DaysoBz_yr6 = pd.read_csv(r'/Users/salinasha/Desktop/MacBoo Codes/Space Physics Codes/Under pySPEDAS_venv/'
'Stable Day List_with Bcomps/pdFile_Day_soBz_withBcomp/pdFile_Day_Yr6_7to24Re_30mFilt_Stable_soBz.txt', sep = '\t')

print('Displaying Dayside(Sept-May)Yr 1-6 SoBz Stable Lists')
print(df_DaysoBz_yr1.dtypes) #displaying column names and type of column data
#show1to6DF(df_DaysoBz_yr1, df_DaysoBz_yr2, df_DaysoBz_yr3, df_DaysoBz_yr4, df_DaysoBz_yr5, df_DaysoBz_yr6)

#Importing Yr1-5 -By Stable List
df_DaynegBy_yr1 = pd.read_csv(r'/Users/salinasha/Desktop/MacBoo Codes/Space Physics Codes/Under pySPEDAS_venv/'
'Stable Day List_with Bcomps/pdFile_Day_negBy_withBcomp/pdFile_Day_Yr1_7to24Re_30mFilt_Stable_negBy.txt', sep = '\t')
df_DaynegBy_yr2 = pd.read_csv(r'/Users/salinasha/Desktop/MacBoo Codes/Space Physics Codes/Under pySPEDAS_venv/'
'Stable Day List_with Bcomps/pdFile_Day_negBy_withBcomp/pdFile_Day_Yr2_7to24Re_30mFilt_Stable_negBy.txt', sep = '\t')
df_DaynegBy_yr3 = pd.read_csv(r'/Users/salinasha/Desktop/MacBoo Codes/Space Physics Codes/Under pySPEDAS_venv/'
'Stable Day List_with Bcomps/pdFile_Day_negBy_withBcomp/pdFile_Day_Yr3_7to24Re_30mFilt_Stable_negBy.txt', sep = '\t')
df_DaynegBy_yr4 = pd.read_csv(r'/Users/salinasha/Desktop/MacBoo Codes/Space Physics Codes/Under pySPEDAS_venv/'
'Stable Day List_with Bcomps/pdFile_Day_negBy_withBcomp/pdFile_Day_Yr4_7to24Re_30mFilt_Stable_negBy.txt', sep = '\t')
df_DaynegBy_yr5 = pd.read_csv(r'/Users/salinasha/Desktop/MacBoo Codes/Space Physics Codes/Under pySPEDAS_venv/'
'Stable Day List_with Bcomps/pdFile_Day_negBy_withBcomp/pdFile_Day_Yr5_7to24Re_30mFilt_Stable_negBy_noMECgap_incr.txt', sep = '\t')
df_DaynegBy_yr6 = pd.read_csv(r'/Users/salinasha/Desktop/MacBoo Codes/Space Physics Codes/Under pySPEDAS_venv/'
'Stable Day List_with Bcomps/pdFile_Day_negBy_withBcomp/pdFile_Day_Yr6_7to24Re_30mFilt_Stable_negBy.txt', sep = '\t')

print('\nDisplaying Dayside(Sept-May)Yr 1-6 NegBy Stable Lists')
show1to6DF(df_DaynegBy_yr1, df_DaynegBy_yr2, df_DaynegBy_yr3, df_DaynegBy_yr4, df_DaynegBy_yr5, df_DaynegBy_yr6)

#Importing Yr1-5 +By Stable List
df_DayposBy_yr1 = pd.read_csv(r'/Users/salinasha/Desktop/MacBoo Codes/Space Physics Codes/Under pySPEDAS_venv/'
'Stable Day List_with Bcomps/pdFile_Day_posBy_withBcomp/pdFile_Day_Yr1_7to24Re_30mFilt_Stable_posBy.txt', sep = '\t')
df_DayposBy_yr2 = pd.read_csv(r'/Users/salinasha/Desktop/MacBoo Codes/Space Physics Codes/Under pySPEDAS_venv/'
'Stable Day List_with Bcomps/pdFile_Day_posBy_withBcomp/pdFile_Day_Yr2_7to24Re_30mFilt_Stable_posBy.txt', sep = '\t')
df_DayposBy_yr3 = pd.read_csv(r'/Users/salinasha/Desktop/MacBoo Codes/Space Physics Codes/Under pySPEDAS_venv/'
'Stable Day List_with Bcomps/pdFile_Day_posBy_withBcomp/pdFile_Day_Yr3_7to24Re_30mFilt_Stable_posBy.txt', sep = '\t')
df_DayposBy_yr4 = pd.read_csv(r'/Users/salinasha/Desktop/MacBoo Codes/Space Physics Codes/Under pySPEDAS_venv/'
'Stable Day List_with Bcomps/pdFile_Day_posBy_withBcomp/pdFile_Day_Yr4_7to24Re_30mFilt_Stable_posBy.txt', sep = '\t')
df_DayposBy_yr5 = pd.read_csv(r'/Users/salinasha/Desktop/MacBoo Codes/Space Physics Codes/Under pySPEDAS_venv/'
'Stable Day List_with Bcomps/pdFile_Day_posBy_withBcomp/pdFile_Day_Yr5_7to24Re_30mFilt_Stable_posBy_noMECgap_incr.txt', sep = '\t')
df_DayposBy_yr6 = pd.read_csv(r'/Users/salinasha/Desktop/MacBoo Codes/Space Physics Codes/Under pySPEDAS_venv/'
'Stable Day List_with Bcomps/pdFile_Day_posBy_withBcomp/pdFile_Day_Yr6_7to24Re_30mFilt_Stable_posBy.txt', sep = '\t')

print('\nDisplaying Dayside(Sept-May)Yr 1-6 PosBy Stable Lists')
#show1to6DF(df_DayposBy_yr1, df_DayposBy_yr2, df_DayposBy_yr3, df_DayposBy_yr4, df_DayposBy_yr5, df_DayposBy_yr6)


#Importing Yr 1-5 NorBz Stable List
df_DaynorBz_yr1 = pd.read_csv(r'/Users/salinasha/Desktop/MacBoo Codes/Space Physics Codes/Under pySPEDAS_venv/'
'Stable Day List_with Bcomps/pdFile_Day_norBz_withBcomp/pdFile_Day_Yr1_7to24Re_30mFilt_Stable_norBz.txt', sep = '\t')
df_DaynorBz_yr2 = pd.read_csv(r'/Users/salinasha/Desktop/MacBoo Codes/Space Physics Codes/Under pySPEDAS_venv/'
'Stable Day List_with Bcomps/pdFile_Day_norBz_withBcomp/pdFile_Day_Yr2_7to24Re_30mFilt_Stable_norBz.txt', sep = '\t')
df_DaynorBz_yr3 = pd.read_csv(r'/Users/salinasha/Desktop/MacBoo Codes/Space Physics Codes/Under pySPEDAS_venv/'
'Stable Day List_with Bcomps/pdFile_Day_norBz_withBcomp/pdFile_Day_Yr3_7to24Re_30mFilt_Stable_norBz.txt', sep = '\t')
df_DaynorBz_yr4 = pd.read_csv(r'/Users/salinasha/Desktop/MacBoo Codes/Space Physics Codes/Under pySPEDAS_venv/'
'Stable Day List_with Bcomps/pdFile_Day_norBz_withBcomp/pdFile_Day_Yr4_7to24Re_30mFilt_Stable_norBz.txt', sep = '\t')
df_DaynorBz_yr5 = pd.read_csv(r'/Users/salinasha/Desktop/MacBoo Codes/Space Physics Codes/Under pySPEDAS_venv/'
'Stable Day List_with Bcomps/pdFile_Day_norBz_withBcomp/pdFile_Day_Yr5_7to24Re_30mFilt_Stable_norBz_noMECgap.txt', sep = '\t')
df_DaynorBz_yr6 = pd.read_csv(r'/Users/salinasha/Desktop/MacBoo Codes/Space Physics Codes/Under pySPEDAS_venv/'
'Stable Day List_with Bcomps/pdFile_Day_norBz_withBcomp/pdFile_Day_Yr6_7to24Re_30mFilt_Stable_norBz.txt', sep = '\t')

print('\nDisplaying Dayside(Sept-May)Yr 1-6 NorBz Stable Lists')
#show1to6DF(df_DaynorBz_yr1, df_DaynorBz_yr2, df_DaynorBz_yr3, df_DaynorBz_yr4, df_DaynorBz_yr5, df_DaynorBz_yr6)

#'''
#Importing Yr 1-5 NonClang Sorted Stable Lists
df_Daystable_yr1 = pd.read_csv(r'/Users/salinasha/Desktop/MacBoo Codes/Space Physics Codes/Under pySPEDAS_venv/'
'Stable Day List_with Bcomps/pdFile_Day_General_withBcomp/pdFile_Day_Yr1_7to24Re_30mFilt_Stable_General.txt', sep = '\t')
df_Daystable_yr2 = pd.read_csv(r'/Users/salinasha/Desktop/MacBoo Codes/Space Physics Codes/Under pySPEDAS_venv/'
'Stable Day List_with Bcomps/pdFile_Day_General_withBcomp/pdFile_Day_Yr2_7to24Re_30mFilt_Stable_General.txt', sep = '\t')
df_Daystable_yr3 = pd.read_csv(r'/Users/salinasha/Desktop/MacBoo Codes/Space Physics Codes/Under pySPEDAS_venv/'
'Stable Day List_with Bcomps/pdFile_Day_General_withBcomp/pdFile_Day_Yr3_7to24Re_30mFilt_Stable_General.txt', sep = '\t')
df_Daystable_yr4 = pd.read_csv(r'/Users/salinasha/Desktop/MacBoo Codes/Space Physics Codes/Under pySPEDAS_venv/'
'Stable Day List_with Bcomps/pdFile_Day_General_withBcomp/pdFile_Day_Yr4_7to24Re_30mFilt_Stable_General.txt', sep = '\t')
df_Daystable_yr5 = pd.read_csv(r'/Users/salinasha/Desktop/MacBoo Codes/Space Physics Codes/Under pySPEDAS_venv/'
'Stable Day List_with Bcomps/pdFile_Day_General_withBcomp/pdFile_Day_Yr5_7to24Re_30mFilt_Stable_General_noMECgap.txt', sep = '\t')
df_Daystable_yr6 = pd.read_csv(r'/Users/salinasha/Desktop/MacBoo Codes/Space Physics Codes/Under pySPEDAS_venv/'
'Stable Day List_with Bcomps/pdFile_Day_General_withBcomp/pdFile_Day_Yr6_7to24Re_30mFilt_Stable_General.txt', sep = '\t')

print('Displaying Dayside(Sept-May)Yr 1-6 General Stable Lists')
#print(df_soBz_yr1.dtypes) #displaying column names and type of column data
#show1to6DF(df_Daystable_yr1, df_Daystable_yr2, df_Daystable_yr3, df_Daystable_yr4, df_Daystable_yr5, df_Daystable_yr6)
#''';



['rmag', 't', 'x', 'y', 'z']
Displaying Dayside(Sept-May)Yr 1-6 SoBz Stable Lists
Start_str      object
Start_ind       int64
Mgs#_avg      float64
Clang_avg     float64
Rmag_avg      float64
Bx_avg(nT)    float64
By_avg(nT)    float64
Bz_avg(nT)    float64
Bdeviat       float64
Dtheta        float64
End_ind         int64
End_str        object
dtype: object

Displaying Dayside(Sept-May)Yr 1-6 NegBy Stable Lists


,Start_str,Start_ind,Mgs#_avg,Clang_avg,Rmag_avg,Bx_avg(nT),By_avg(nT),Bz_avg(nT),Bdeviat,Dtheta,End_ind,End_str
0,2015-09-01 00:10:00,2,5.28,270.271,8.646,3.502,-3.177,0.023,0.08,5.28,7,2015-09-01 00:35:00
1,2015-09-01 08:40:00,88,5.42,284.483,7.927,1.962,-4.547,1.157,0.02,27.44,93,2015-09-01 09:05:00
2,2015-09-01 10:35:00,111,5.33,260.121,9.749,3.143,-3.738,-0.730,0.02,27.20,116,2015-09-01 11:00:00
3,2015-09-01 12:30:00,134,5.17,299.448,10.965,3.513,-3.783,2.150,0.04,12.78,139,2015-09-01 12:55:00
4,2015-09-01 19:15:00,215,5.38,251.441,11.616,3.822,-3.078,-1.018,0.03,14.60,220,2015-09-01 19:40:00
...,...,...,...,...,...,...,...,...,...,...,...,...
660,2016-05-30 09:05:00,72631,6.57,271.550,11.994,2.197,-4.147,0.128,0.07,9.33,72636,2016-05-30 09:30:00
661,2016-05-30 09:35:00,72637,6.13,275.711,12.009,3.940,-3.460,0.367,0.06,15.26,72642,2016-05-30 10:00:00
662,2016-05-30 10:05:00,72643,6.17,267.143,11.996,3.412,-4.288,-0.208,0.04,5.79,72648,2016-05-30 10:30:00
663,2016-05-30 13:05:00,72679,6.92,266.897,11.312,3.228,-1.163,-0.033,0.09,12.47,72684,2016-05-30 13:30:00


,Start_str,Start_ind,Mgs#_avg,Clang_avg,Rmag_avg,Bx_avg(nT),By_avg(nT),Bz_avg(nT),Bdeviat,Dtheta,End_ind,End_str
0,2016-09-08 16:00:00,1935,6.93,254.787,9.922,-1.043,-4.162,-1.125,0.04,19.62,1940,2016-09-08 16:25:00
1,2016-09-11 22:15:00,2854,5.65,240.055,11.952,-2.332,-4.287,-2.442,0.02,13.50,2859,2016-09-11 22:40:00
2,2016-09-11 22:45:00,2860,5.72,237.653,11.897,-4.373,-3.188,-2.063,0.07,22.09,2865,2016-09-11 23:10:00
3,2016-09-16 13:40:00,4191,5.03,305.148,8.705,2.902,-4.585,3.210,0.05,17.18,4196,2016-09-16 14:05:00
4,2016-09-17 17:25:00,4524,5.60,302.025,11.303,2.752,-2.783,1.733,0.05,7.31,4529,2016-09-17 17:50:00
...,...,...,...,...,...,...,...,...,...,...,...,...
608,2017-05-27 10:30:00,73252,5.78,274.884,8.388,-0.223,-2.595,0.222,0.08,22.23,73257,2017-05-27 10:55:00
609,2017-05-27 11:00:00,73258,5.88,278.537,9.165,-0.262,-2.617,0.395,0.01,21.20,73263,2017-05-27 11:25:00
610,2017-05-27 11:30:00,73264,5.90,268.346,9.897,0.222,-2.638,-0.082,0.07,24.93,73269,2017-05-27 11:55:00
611,2017-05-27 14:15:00,73297,5.87,264.697,13.330,-1.552,-2.630,-0.240,0.10,25.11,73302,2017-05-27 14:40:00


,Start_str,Start_ind,Mgs#_avg,Clang_avg,Rmag_avg,Bx_avg(nT),By_avg(nT),Bz_avg(nT),Bdeviat,Dtheta,End_ind,End_str
0,2017-09-01 18:35:00,223,6.20,311.273,18.642,-4.432,-3.587,3.130,0.03,29.17,228,2017-09-01 19:00:00
1,2017-09-01 20:30:00,246,6.37,279.942,17.200,-5.642,-2.452,0.287,0.08,25.32,251,2017-09-01 20:55:00
2,2017-09-01 22:25:00,259,6.03,310.823,15.542,-5.735,-3.620,3.128,0.06,21.42,264,2017-09-01 22:50:00
3,2017-09-03 05:30:00,631,6.95,309.260,23.015,-3.452,-1.092,0.905,0.08,26.70,636,2017-09-03 05:55:00
4,2017-09-06 22:45:00,1699,6.47,295.288,23.593,-0.320,-2.987,1.430,0.08,19.97,1704,2017-09-06 23:10:00
...,...,...,...,...,...,...,...,...,...,...,...,...
817,2018-05-22 14:15:00,71970,5.70,265.307,17.828,4.483,-5.903,-0.462,0.09,22.07,71975,2018-05-22 14:40:00
818,2018-05-22 14:45:00,71976,5.72,265.247,18.202,4.557,-5.773,-0.492,0.04,17.35,71981,2018-05-22 15:10:00
819,2018-05-22 16:15:00,71994,5.67,271.157,19.249,4.250,-6.382,0.187,0.09,28.11,71999,2018-05-22 16:40:00
820,2018-05-24 17:35:00,72586,6.90,263.469,9.301,-3.033,-0.715,-0.083,0.10,18.64,72591,2018-05-24 18:00:00


,Start_str,Start_ind,Mgs#_avg,Clang_avg,Rmag_avg,Bx_avg(nT),By_avg(nT),Bz_avg(nT),Bdeviat,Dtheta,End_ind,End_str
0,2018-09-01 01:20:00,16,6.43,284.177,18.733,-3.255,-1.057,0.307,0.05,23.74,21,2018-09-01 01:45:00
1,2018-09-01 03:45:00,45,6.48,312.375,20.288,-2.530,-1.597,1.462,0.05,13.52,50,2018-09-01 04:10:00
2,2018-09-02 09:00:00,396,6.72,308.625,23.407,1.515,-1.805,1.445,0.05,5.62,401,2018-09-02 09:25:00
3,2018-09-02 09:30:00,402,6.60,313.785,23.235,1.790,-1.483,1.420,0.06,9.89,407,2018-09-02 09:55:00
4,2018-09-02 11:30:00,426,6.12,302.135,22.464,1.747,-2.313,1.455,0.06,25.33,431,2018-09-02 11:55:00
...,...,...,...,...,...,...,...,...,...,...,...,...
892,2019-05-24 12:40:00,72182,5.98,278.693,14.310,1.618,-3.207,0.517,0.07,27.07,72187,2019-05-24 13:05:00
893,2019-05-24 13:10:00,72188,6.20,265.719,13.749,1.315,-3.248,-0.265,0.09,22.99,72193,2019-05-24 13:35:00
894,2019-05-28 15:55:00,73373,5.15,299.266,13.273,0.162,-6.640,3.740,0.07,12.04,73378,2019-05-28 16:20:00
895,2019-05-28 20:30:00,73428,5.05,236.574,17.884,5.833,-4.823,-3.205,0.10,13.43,73433,2019-05-28 20:55:00


,Start_str,Start_ind,Mgs#_avg,Clang_avg,Rmag_avg,Bx_avg(nT),By_avg(nT),Bz_avg(nT),Bdeviat,Dtheta,End_ind,End_str
0,2019-09-17 07:15:00,4513,5.88,284.799,9.618,-0.280,-4.797,1.262,0.06,7.48,4518,2019-09-17 07:40:00
1,2019-09-17 15:00:00,4606,5.13,294.556,7.924,2.822,-3.960,1.790,0.03,9.21,4611,2019-09-17 15:25:00
2,2019-09-17 15:30:00,4612,5.03,294.403,8.730,2.950,-3.825,1.777,0.02,22.56,4617,2019-09-17 15:55:00
3,2019-09-17 16:25:00,4623,5.12,235.839,10.102,3.665,-2.655,-1.760,0.06,23.84,4628,2019-09-17 16:50:00
4,2019-09-20 09:35:00,5404,6.17,251.279,19.678,-1.085,-2.295,-0.712,0.04,24.22,5409,2019-09-20 10:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...
527,2020-05-30 10:35:00,71251,5.23,267.097,13.176,5.738,-10.587,-0.558,0.04,20.43,71256,2020-05-30 11:00:00
528,2020-05-30 11:05:00,71257,5.37,270.261,13.730,5.477,-10.423,0.052,0.05,7.24,71262,2020-05-30 11:30:00
529,2020-05-30 12:00:00,71268,5.37,278.151,14.696,4.732,-9.122,1.333,0.09,11.86,71273,2020-05-30 12:25:00
530,2020-05-30 13:20:00,71284,5.57,288.296,16.002,4.653,-8.377,2.793,0.09,25.20,71289,2020-05-30 13:45:00


,Start_str,Start_ind,Mgs#_avg,Clang_avg,Rmag_avg,Bx_avg(nT),By_avg(nT),Bz_avg(nT),Bdeviat,Dtheta,End_ind,End_str
0,2020-09-04 11:55:00,1006,5.82,230.346,16.924,-1.595,-3.253,-2.697,0.03,3.61,1011,2020-09-04 12:20:00
1,2020-09-05 11:30:00,1287,5.78,278.429,17.760,-2.863,-3.333,0.495,0.09,16.08,1292,2020-09-05 11:55:00
2,2020-09-05 12:05:00,1294,5.88,280.664,18.234,-1.750,-4.168,0.778,0.02,15.71,1299,2020-09-05 12:30:00
3,2020-09-05 13:50:00,1315,5.70,306.456,19.565,-2.135,-3.267,2.450,0.05,28.62,1320,2020-09-05 14:15:00
4,2020-09-05 14:20:00,1321,5.73,289.126,19.921,-1.967,-3.955,1.370,0.05,23.20,1326,2020-09-05 14:45:00
...,...,...,...,...,...,...,...,...,...,...,...,...
637,2021-05-25 12:00:00,73973,5.80,269.120,11.525,0.857,-3.748,-0.070,0.06,16.75,73978,2021-05-25 12:25:00
638,2021-05-25 12:40:00,73981,6.40,286.236,10.695,1.723,-2.167,0.625,0.02,10.34,73986,2021-05-25 13:05:00
639,2021-05-25 21:55:00,74089,5.92,228.189,7.845,0.762,-2.085,-1.870,0.10,11.83,74094,2021-05-25 22:20:00
640,2021-05-26 01:55:00,74137,6.68,235.480,13.013,1.132,-1.710,-1.178,0.08,5.03,74142,2021-05-26 02:20:00



Displaying Dayside(Sept-May)Yr 1-6 PosBy Stable Lists

Displaying Dayside(Sept-May)Yr 1-6 NorBz Stable Lists
Displaying Dayside(Sept-May)Yr 1-6 General Stable Lists


#### Importing Nightside(Jun to Aug) Yrs1-7 Stable IMF Lists

In [3]:
#'''
#Load Nightside MEC Data:   Extracting MEC radial position(km) and time(unix time) arrays
night_yr1_MECposdata = np.load(r'/Users/salinasha/Desktop/MacBoo Codes/Space Physics Codes/Under pySPEDAS_venv/MEC exported Nightside posdata/Nightside_yr1_MECgsm_posdata.npz')
night_yr2_MECposdata = np.load(r'/Users/salinasha/Desktop/MacBoo Codes/Space Physics Codes/Under pySPEDAS_venv/MEC exported Nightside posdata/Nightside_yr2_MECgsm_posdata.npz')
night_yr3_MECposdata = np.load(r'/Users/salinasha/Desktop/MacBoo Codes/Space Physics Codes/Under pySPEDAS_venv/MEC exported Nightside posdata/Nightside_yr3_MECgsm_posdata.npz')
night_yr4_MECposdata = np.load(r'/Users/salinasha/Desktop/MacBoo Codes/Space Physics Codes/Under pySPEDAS_venv/MEC exported Nightside posdata/Nightside_yr4_MECgsm_posdata.npz')
night_yr5_MECposdata = np.load(r'/Users/salinasha/Desktop/MacBoo Codes/Space Physics Codes/Under pySPEDAS_venv/MEC exported Nightside posdata/Nightside_yr5_MECgsm_posdata.npz')
night_yr6_MECposdata = np.load(r'/Users/salinasha/Desktop/MacBoo Codes/Space Physics Codes/Under pySPEDAS_venv/MEC exported Nightside posdata/Nightside_yr6_MECgsm_posdata.npz')
night_yr7_MECposdata = np.load(r'/Users/salinasha/Desktop/MacBoo Codes/Space Physics Codes/Under pySPEDAS_venv/MEC exported Nightside posdata/Nightside_yr7_MECgsm_posdata.npz')


#Importing Night Yr 1-5 SoBZ Stable List##
df_NightsoBz_yr1 = pd.read_csv(r'/Users/salinasha/Desktop/MacBoo Codes/Space Physics Codes/'
'Under pySPEDAS_venv/Night_Stable IMF Lists/pdFile_Night_soBz/pdFile_Night_Yr1_7to24Re_30mFilt_Stable_soBz.txt', sep = '\t')
df_NightsoBz_yr2 = pd.read_csv(r'/Users/salinasha/Desktop/MacBoo Codes/Space Physics Codes/'
'Under pySPEDAS_venv/Night_Stable IMF Lists/pdFile_Night_soBz/pdFile_Night_Yr2_7to24Re_30mFilt_Stable_soBz.txt', sep = '\t')
df_NightsoBz_yr3 = pd.read_csv(r'/Users/salinasha/Desktop/MacBoo Codes/Space Physics Codes/'
'Under pySPEDAS_venv/Night_Stable IMF Lists/pdFile_Night_soBz/pdFile_Night_Yr3_7to24Re_30mFilt_Stable_soBz.txt', sep = '\t')
df_NightsoBz_yr4 = pd.read_csv(r'/Users/salinasha/Desktop/MacBoo Codes/Space Physics Codes/'
'Under pySPEDAS_venv/Night_Stable IMF Lists/pdFile_Night_soBz/pdFile_Night_Yr4_7to24Re_30mFilt_Stable_soBz.txt', sep = '\t')
df_NightsoBz_yr5 = pd.read_csv(r'/Users/salinasha/Desktop/MacBoo Codes/Space Physics Codes/'
'Under pySPEDAS_venv/Night_Stable IMF Lists/pdFile_Night_soBz/pdFile_Night_Yr5_7to24Re_30mFilt_Stable_soBz.txt', sep = '\t')
df_NightsoBz_yr6 = pd.read_csv(r'/Users/salinasha/Desktop/MacBoo Codes/Space Physics Codes/'
'Under pySPEDAS_venv/Night_Stable IMF Lists/pdFile_Night_soBz/pdFile_Night_Yr6_7to24Re_30mFilt_Stable_soBz.txt', sep = '\t')
df_NightsoBz_yr7 = pd.read_csv(r'/Users/salinasha/Desktop/MacBoo Codes/Space Physics Codes/'
'Under pySPEDAS_venv/Night_Stable IMF Lists/pdFile_Night_soBz/pdFile_Night_Yr7_7to24Re_30mFilt_Stable_soBz.txt', sep = '\t')

print('Displaying Nightside(Jun-Aug)Yr 1-7 SoBz Stable Lists')
print(df_NightsoBz_yr1.dtypes) #displaying column names and type of column data
#show1to5DF(df_NightsoBz_yr1, df_NightsoBz_yr2, df_NightsoBz_yr3, df_NightsoBz_yr4, df_NightsoBz_yr5)
#display(df_NightsoBz_yr6)
#display(df_NightsoBz_yr7)

#Importing Night Yr 1-7 norBz Stable List##
df_NightnorBz_yr1 = pd.read_csv(r'/Users/salinasha/Desktop/MacBoo Codes/Space Physics Codes/'
'Under pySPEDAS_venv/Night_Stable IMF Lists/pdFile_Night_norBz/pdFile_Night_Yr1_7to24Re_30mFilt_Stable_norBz.txt', sep = '\t')
df_NightnorBz_yr2 = pd.read_csv(r'/Users/salinasha/Desktop/MacBoo Codes/Space Physics Codes/'
'Under pySPEDAS_venv/Night_Stable IMF Lists/pdFile_Night_norBz/pdFile_Night_Yr2_7to24Re_30mFilt_Stable_norBz.txt', sep = '\t')
df_NightnorBz_yr3 = pd.read_csv(r'/Users/salinasha/Desktop/MacBoo Codes/Space Physics Codes/'
'Under pySPEDAS_venv/Night_Stable IMF Lists/pdFile_Night_norBz/pdFile_Night_Yr3_7to24Re_30mFilt_Stable_norBz.txt', sep = '\t')
df_NightnorBz_yr4 = pd.read_csv(r'/Users/salinasha/Desktop/MacBoo Codes/Space Physics Codes/'
'Under pySPEDAS_venv/Night_Stable IMF Lists/pdFile_Night_norBz/pdFile_Night_Yr4_7to24Re_30mFilt_Stable_norBz.txt', sep = '\t')
df_NightnorBz_yr5 = pd.read_csv(r'/Users/salinasha/Desktop/MacBoo Codes/Space Physics Codes/'
'Under pySPEDAS_venv/Night_Stable IMF Lists/pdFile_Night_norBz/pdFile_Night_Yr5_7to24Re_30mFilt_Stable_norBz.txt', sep = '\t')
df_NightnorBz_yr6 = pd.read_csv(r'/Users/salinasha/Desktop/MacBoo Codes/Space Physics Codes/'
'Under pySPEDAS_venv/Night_Stable IMF Lists/pdFile_Night_norBz/pdFile_Night_Yr6_7to24Re_30mFilt_Stable_norBz.txt', sep = '\t')
df_NightnorBz_yr7 = pd.read_csv(r'/Users/salinasha/Desktop/MacBoo Codes/Space Physics Codes/'
'Under pySPEDAS_venv/Night_Stable IMF Lists/pdFile_Night_norBz/pdFile_Night_Yr7_7to24Re_30mFilt_Stable_norBz.txt', sep = '\t')

print('Displaying Nightside(Jun-Aug)Yr 1-7 norBz Stable Lists')
#show1to5DF(df_NightnorBz_yr1, df_NightnorBz_yr2, df_NightnorBz_yr3, df_NightnorBz_yr4, df_NightnorBz_yr5)
#display(df_NightnorBz_yr6)
#display(df_NightnorBz_yr7)

#Importing Night Yr 1-7 posBy Stable List##
df_NightposBy_yr1 = pd.read_csv(r'/Users/salinasha/Desktop/MacBoo Codes/Space Physics Codes/'
'Under pySPEDAS_venv/Night_Stable IMF Lists/pdFile_Night_posBy/pdFile_Night_Yr1_7to24Re_30mFilt_Stable_posBy.txt', sep = '\t')
df_NightposBy_yr2 = pd.read_csv(r'/Users/salinasha/Desktop/MacBoo Codes/Space Physics Codes/'
'Under pySPEDAS_venv/Night_Stable IMF Lists/pdFile_Night_posBy/pdFile_Night_Yr2_7to24Re_30mFilt_Stable_posBy.txt', sep = '\t')
df_NightposBy_yr3 = pd.read_csv(r'/Users/salinasha/Desktop/MacBoo Codes/Space Physics Codes/'
'Under pySPEDAS_venv/Night_Stable IMF Lists/pdFile_Night_posBy/pdFile_Night_Yr3_7to24Re_30mFilt_Stable_posBy.txt', sep = '\t')
df_NightposBy_yr4 = pd.read_csv(r'/Users/salinasha/Desktop/MacBoo Codes/Space Physics Codes/'
'Under pySPEDAS_venv/Night_Stable IMF Lists/pdFile_Night_posBy/pdFile_Night_Yr4_7to24Re_30mFilt_Stable_posBy.txt', sep = '\t')
df_NightposBy_yr5 = pd.read_csv(r'/Users/salinasha/Desktop/MacBoo Codes/Space Physics Codes/'
'Under pySPEDAS_venv/Night_Stable IMF Lists/pdFile_Night_posBy/pdFile_Night_Yr5_7to24Re_30mFilt_Stable_posBy.txt', sep = '\t')
df_NightposBy_yr6 = pd.read_csv(r'/Users/salinasha/Desktop/MacBoo Codes/Space Physics Codes/'
'Under pySPEDAS_venv/Night_Stable IMF Lists/pdFile_Night_posBy/pdFile_Night_Yr6_7to24Re_30mFilt_Stable_posBy.txt', sep = '\t')
df_NightposBy_yr7 = pd.read_csv(r'/Users/salinasha/Desktop/MacBoo Codes/Space Physics Codes/'
'Under pySPEDAS_venv/Night_Stable IMF Lists/pdFile_Night_posBy/pdFile_Night_Yr7_7to24Re_30mFilt_Stable_posBy.txt', sep = '\t')

print('Displaying Nightside(Jun-Aug)Yr 1-7 posBy Stable Lists')
show1to5DF(df_NightposBy_yr1, df_NightposBy_yr2, df_NightposBy_yr3, df_NightposBy_yr4, df_NightposBy_yr5)
display(df_NightposBy_yr6)
display(df_NightposBy_yr7)

#Importing Night Yr 1-7 negBy Stable List##
df_NightnegBy_yr1 = pd.read_csv(r'/Users/salinasha/Desktop/MacBoo Codes/Space Physics Codes/'
'Under pySPEDAS_venv/Night_Stable IMF Lists/pdFile_Night_negBy/pdFile_Night_Yr1_7to24Re_30mFilt_Stable_negBy.txt', sep = '\t')
df_NightnegBy_yr2 = pd.read_csv(r'/Users/salinasha/Desktop/MacBoo Codes/Space Physics Codes/'
'Under pySPEDAS_venv/Night_Stable IMF Lists/pdFile_Night_negBy/pdFile_Night_Yr2_7to24Re_30mFilt_Stable_negBy.txt', sep = '\t')
df_NightnegBy_yr3 = pd.read_csv(r'/Users/salinasha/Desktop/MacBoo Codes/Space Physics Codes/'
'Under pySPEDAS_venv/Night_Stable IMF Lists/pdFile_Night_negBy/pdFile_Night_Yr3_7to24Re_30mFilt_Stable_negBy.txt', sep = '\t')
df_NightnegBy_yr4 = pd.read_csv(r'/Users/salinasha/Desktop/MacBoo Codes/Space Physics Codes/'
'Under pySPEDAS_venv/Night_Stable IMF Lists/pdFile_Night_negBy/pdFile_Night_Yr4_7to24Re_30mFilt_Stable_negBy.txt', sep = '\t')
df_NightnegBy_yr5 = pd.read_csv(r'/Users/salinasha/Desktop/MacBoo Codes/Space Physics Codes/'
'Under pySPEDAS_venv/Night_Stable IMF Lists/pdFile_Night_negBy/pdFile_Night_Yr5_7to24Re_30mFilt_Stable_negBy.txt', sep = '\t')
df_NightnegBy_yr6 = pd.read_csv(r'/Users/salinasha/Desktop/MacBoo Codes/Space Physics Codes/'
'Under pySPEDAS_venv/Night_Stable IMF Lists/pdFile_Night_negBy/pdFile_Night_Yr6_7to24Re_30mFilt_Stable_negBy.txt', sep = '\t')
df_NightnegBy_yr7 = pd.read_csv(r'/Users/salinasha/Desktop/MacBoo Codes/Space Physics Codes/'
'Under pySPEDAS_venv/Night_Stable IMF Lists/pdFile_Night_negBy/pdFile_Night_Yr7_7to24Re_30mFilt_Stable_negBy.txt', sep = '\t')

print('Displaying Nightside(Jun-Aug)Yr 1-7 negBy Stable Lists')
show1to5DF(df_NightnegBy_yr1, df_NightnegBy_yr2, df_NightnegBy_yr3, df_NightnegBy_yr4, df_NightnegBy_yr5)
display(df_NightnegBy_yr6)
display(df_NightnegBy_yr7)

#Importing Yr 1-7 NonClang Sorted Stable Lists
df_Nightstable_yr1 = pd.read_csv(r'/Users/salinasha/Desktop/MacBoo Codes/Space Physics Codes/'
'Under pySPEDAS_venv/Night_Stable IMF Lists/pdFile_Night_General/pdFile_Night_Yr1_7to24Re_30mFilt_Stable_General.txt', sep = '\t')
df_Nightstable_yr2 = pd.read_csv(r'/Users/salinasha/Desktop/MacBoo Codes/Space Physics Codes/'
'Under pySPEDAS_venv/Night_Stable IMF Lists/pdFile_Night_General/pdFile_Night_Yr2_7to24Re_30mFilt_Stable_General.txt', sep = '\t')
df_Nightstable_yr3 = pd.read_csv(r'/Users/salinasha/Desktop/MacBoo Codes/Space Physics Codes/'
'Under pySPEDAS_venv/Night_Stable IMF Lists/pdFile_Night_General/pdFile_Night_Yr3_7to24Re_30mFilt_Stable_General.txt', sep = '\t')
df_Nightstable_yr4 = pd.read_csv(r'/Users/salinasha/Desktop/MacBoo Codes/Space Physics Codes/'
'Under pySPEDAS_venv/Night_Stable IMF Lists/pdFile_Night_General/pdFile_Night_Yr4_7to24Re_30mFilt_Stable_General.txt', sep = '\t')
df_Nightstable_yr5 = pd.read_csv(r'/Users/salinasha/Desktop/MacBoo Codes/Space Physics Codes/'
'Under pySPEDAS_venv/Night_Stable IMF Lists/pdFile_Night_General/pdFile_Night_Yr5_7to24Re_30mFilt_Stable_General.txt', sep = '\t')
df_Nightstable_yr6 = pd.read_csv(r'/Users/salinasha/Desktop/MacBoo Codes/Space Physics Codes/'
'Under pySPEDAS_venv/Night_Stable IMF Lists/pdFile_Night_General/pdFile_Night_Yr6_7to24Re_30mFilt_Stable_General.txt', sep = '\t')
df_Nightstable_yr7 = pd.read_csv(r'/Users/salinasha/Desktop/MacBoo Codes/Space Physics Codes/'
'Under pySPEDAS_venv/Night_Stable IMF Lists/pdFile_Night_General/pdFile_Night_Yr7_7to24Re_30mFilt_Stable_General.txt', sep = '\t')

print('Displaying Nightside Yr 1-7 General Stable Lists')
#show1to5DF(df_Nightstable_yr1, df_Nightstable_yr2, df_Nightstable_yr3, df_Nightstable_yr4, df_Nightstable_yr5)
#display(df_Nightstable_yr6)
#display(df_Nightstable_yr7)
#''';


Displaying Nightside(Jun-Aug)Yr 1-7 SoBz Stable Lists
Start_str      object
Start_ind       int64
Mgs#_avg      float64
Clang_avg     float64
Rmag_avg      float64
Bx_avg(nT)    float64
By_avg(nT)    float64
Bz_avg(nT)    float64
Bdeviat       float64
Dtheta        float64
End_ind         int64
End_str        object
dtype: object
Displaying Nightside(Jun-Aug)Yr 1-7 norBz Stable Lists
Displaying Nightside(Jun-Aug)Yr 1-7 posBy Stable Lists


,Start_str,Start_ind,Mgs#_avg,Clang_avg,Rmag_avg,Bx_avg(nT),By_avg(nT),Bz_avg(nT),Bdeviat,Dtheta,End_ind,End_str
0,2015-06-01 07:30:00,90,5.02,112.984,10.912,-3.340,4.468,-1.863,0.03,18.25,95,2015-06-01 07:55:00
1,2015-06-01 08:00:00,96,5.10,93.657,10.634,-1.402,5.523,-0.335,0.01,13.42,101,2015-06-01 08:25:00
2,2015-06-01 10:30:00,126,5.03,82.656,8.670,-3.748,3.315,0.452,0.05,24.08,131,2015-06-01 10:55:00
3,2015-06-01 21:15:00,243,5.12,56.757,10.035,-0.642,4.498,2.995,0.09,26.53,248,2015-06-01 21:40:00
4,2015-06-02 03:30:00,318,5.05,47.210,12.009,0.610,4.537,4.170,0.09,18.98,323,2015-06-02 03:55:00
...,...,...,...,...,...,...,...,...,...,...,...,...
394,2015-08-26 01:45:00,22470,5.80,100.515,7.664,3.415,6.328,-1.225,0.06,18.33,22475,2015-08-26 02:10:00
395,2015-08-26 02:15:00,22476,5.72,103.717,7.034,2.693,6.778,-1.738,0.05,29.79,22481,2015-08-26 02:40:00
396,2015-08-26 10:00:00,22530,5.45,126.521,8.618,4.793,6.402,-4.810,0.03,28.87,22535,2015-08-26 10:25:00
397,2015-08-26 10:30:00,22536,5.48,120.923,9.097,5.012,6.795,-4.067,0.03,6.13,22541,2015-08-26 10:55:00


,Start_str,Start_ind,Mgs#_avg,Clang_avg,Rmag_avg,Bx_avg(nT),By_avg(nT),Bz_avg(nT),Bdeviat,Dtheta,End_ind,End_str
0,2016-06-02 00:30:00,293,5.38,57.640,7.045,4.173,1.298,0.803,0.09,16.64,298,2016-06-02 00:55:00
1,2016-06-05 03:35:00,1144,5.88,115.085,10.329,-1.730,5.193,-2.432,0.09,25.72,1149,2016-06-05 04:00:00
2,2016-06-09 09:55:00,2363,5.53,53.073,11.896,3.258,1.760,1.318,0.09,5.56,2368,2016-06-09 10:20:00
3,2016-06-10 16:00:00,2724,5.32,78.493,8.385,-3.650,6.497,1.327,0.08,14.46,2729,2016-06-10 16:25:00
4,2016-06-10 16:30:00,2730,5.12,72.778,7.829,-3.775,7.142,2.238,0.10,16.21,2735,2016-06-10 16:55:00
...,...,...,...,...,...,...,...,...,...,...,...,...
230,2016-08-28 14:40:00,22632,6.08,53.856,7.346,2.382,2.430,1.788,0.05,27.54,22637,2016-08-28 15:05:00
231,2016-08-29 17:25:00,22881,5.70,128.040,10.092,-3.610,2.393,-1.920,0.05,27.10,22886,2016-08-29 17:50:00
232,2016-08-30 03:25:00,22951,5.00,134.520,10.878,-6.558,2.717,-2.707,0.07,25.62,22956,2016-08-30 03:50:00
233,2016-08-30 19:05:00,23101,5.72,54.188,11.119,-1.313,6.098,4.440,0.05,17.72,23106,2016-08-30 19:30:00


,Start_str,Start_ind,Mgs#_avg,Clang_avg,Rmag_avg,Bx_avg(nT),By_avg(nT),Bz_avg(nT),Bdeviat,Dtheta,End_ind,End_str
0,2017-06-01 00:00:00,0,5.12,66.934,22.907,-2.442,6.127,2.598,0.07,15.50,5,2017-06-01 00:25:00
1,2017-06-01 00:30:00,6,5.42,62.966,22.718,-1.187,6.173,3.150,0.03,6.66,11,2017-06-01 00:55:00
2,2017-06-01 01:00:00,12,5.35,69.270,22.521,-1.030,6.558,2.468,0.01,10.92,17,2017-06-01 01:25:00
3,2017-06-01 01:30:00,18,5.22,66.155,22.314,-1.557,6.302,2.783,0.01,6.43,23,2017-06-01 01:55:00
4,2017-06-01 02:00:00,24,5.28,61.896,22.099,-3.812,5.147,2.735,0.05,5.93,29,2017-06-01 02:25:00
...,...,...,...,...,...,...,...,...,...,...,...,...
238,2017-08-25 15:20:00,22756,5.37,91.360,20.993,-2.992,5.058,-0.135,0.05,11.95,22761,2017-08-25 15:45:00
239,2017-08-25 16:25:00,22769,5.48,120.968,21.543,-2.938,3.703,-2.227,0.03,29.89,22774,2017-08-25 16:50:00
240,2017-08-25 16:55:00,22775,5.57,133.794,21.781,-3.352,2.652,-2.545,0.08,7.84,22780,2017-08-25 17:20:00
241,2017-08-25 17:25:00,22781,5.62,128.043,22.010,-3.207,2.863,-2.233,0.02,14.03,22786,2017-08-25 17:50:00


,Start_str,Start_ind,Mgs#_avg,Clang_avg,Rmag_avg,Bx_avg(nT),By_avg(nT),Bz_avg(nT),Bdeviat,Dtheta,End_ind,End_str
0,2018-06-08 21:10:00,2169,6.90,83.417,23.012,-2.085,2.532,0.287,0.05,8.26,2174,2018-06-08 21:35:00
1,2018-06-08 22:25:00,2184,6.43,93.096,23.449,-0.808,3.412,-0.190,0.06,6.65,2189,2018-06-08 22:50:00
2,2018-06-08 23:00:00,2191,6.45,94.207,23.636,-0.780,3.268,-0.245,0.09,9.02,2196,2018-06-08 23:25:00
3,2018-06-08 23:45:00,2200,6.62,110.452,23.860,0.328,2.725,-1.018,0.09,10.27,2205,2018-06-09 00:10:00
4,2018-06-09 20:00:00,2443,6.60,92.703,23.601,-1.703,2.180,-0.102,0.10,14.42,2448,2018-06-09 20:25:00
...,...,...,...,...,...,...,...,...,...,...,...,...
220,2018-08-27 22:00:00,23281,6.77,58.448,21.745,-3.187,2.435,1.547,0.06,26.42,23286,2018-08-27 22:25:00
221,2018-08-27 22:30:00,23287,6.70,47.403,21.500,-3.483,2.200,2.020,0.03,15.43,23292,2018-08-27 22:55:00
222,2018-08-27 23:00:00,23293,6.82,67.745,21.245,-3.095,3.182,1.307,0.05,22.69,23298,2018-08-27 23:25:00
223,2018-08-28 22:30:00,23575,6.98,59.626,11.967,-3.060,1.618,0.882,0.03,26.65,23580,2018-08-28 22:55:00


...

,Start_str,Start_ind,Mgs#_avg,Clang_avg,Rmag_avg,Bx_avg(nT),By_avg(nT),Bz_avg(nT),Bdeviat,Dtheta,End_ind,End_str
0,2018-06-02 17:05:00,493,6.38,304.109,14.843,4.523,-3.513,2.282,0.02,29.93,498,2018-06-02 17:30:00
1,2018-06-04 10:25:00,892,6.68,277.666,21.348,3.815,-1.967,0.257,0.06,24.03,897,2018-06-04 10:50:00
2,2018-06-04 12:20:00,915,6.72,294.454,20.296,4.328,-1.463,0.662,0.04,12.47,920,2018-06-04 12:45:00
3,2018-06-04 12:50:00,921,6.72,288.418,19.997,3.978,-1.982,0.663,0.06,21.87,926,2018-06-04 13:15:00
4,2018-06-04 13:20:00,927,6.15,286.128,19.686,3.717,-2.055,0.595,0.05,10.48,932,2018-06-04 13:45:00
...,...,...,...,...,...,...,...,...,...,...,...,...
284,2018-08-30 20:05:00,24121,5.70,266.572,20.389,-1.180,-4.237,-0.265,0.02,14.84,24126,2018-08-30 20:30:00
285,2018-08-30 20:40:00,24128,5.35,267.789,20.042,-1.267,-4.523,-0.202,0.08,13.71,24133,2018-08-30 21:05:00
286,2018-08-30 21:10:00,24134,5.17,255.979,19.733,0.000,-4.782,-1.195,0.03,10.19,24139,2018-08-30 21:35:00
287,2018-08-30 22:30:00,24150,6.92,239.805,18.852,1.937,-2.883,-1.672,0.09,19.43,24155,2018-08-30 22:55:00


,Start_str,Start_ind,Mgs#_avg,Clang_avg,Rmag_avg,Bx_avg(nT),By_avg(nT),Bz_avg(nT),Bdeviat,Dtheta,End_ind,End_str
0,2019-06-01 00:05:00,1,6.32,257.237,8.075,1.465,-1.693,-0.392,0.10,8.21,6,2019-06-01 00:30:00
1,2019-06-01 06:20:00,73,5.70,251.751,15.959,1.675,-2.445,-0.813,0.06,8.50,78,2019-06-01 06:45:00
2,2019-06-01 12:05:00,142,5.33,281.045,20.735,1.910,-1.903,0.380,0.08,21.85,147,2019-06-01 12:30:00
3,2019-06-01 12:35:00,148,5.17,278.571,21.079,2.458,-1.438,0.210,0.04,20.96,153,2019-06-01 13:00:00
4,2019-06-01 13:05:00,154,5.22,273.090,21.413,2.258,-1.897,0.097,0.02,6.67,159,2019-06-01 13:30:00
...,...,...,...,...,...,...,...,...,...,...,...,...
227,2019-08-27 22:15:00,22649,6.92,254.653,17.476,-2.693,-3.213,-0.863,0.06,20.53,22654,2019-08-27 22:40:00
228,2019-08-30 03:05:00,23204,6.65,262.737,23.616,2.083,-2.355,-0.275,0.10,26.81,23209,2019-08-30 03:30:00
229,2019-08-30 03:45:00,23212,6.43,308.751,23.255,2.253,-1.945,1.552,0.08,15.45,23217,2019-08-30 04:10:00
230,2019-08-30 12:30:00,23317,6.30,240.611,16.970,4.650,-4.623,-2.702,0.08,27.65,23322,2019-08-30 12:55:00


,Start_str,Start_ind,Mgs#_avg,Clang_avg,Rmag_avg,Bx_avg(nT),By_avg(nT),Bz_avg(nT),Bdeviat,Dtheta,End_ind,End_str
0,2020-06-02 06:55:00,319,5.00,226.182,13.729,3.943,-3.213,-3.013,0.04,26.45,324,2020-06-02 07:20:00
1,2020-06-02 09:00:00,344,5.27,226.166,11.284,1.592,-4.300,-4.105,0.05,20.13,349,2020-06-02 09:25:00
2,2020-06-02 10:00:00,356,5.38,229.107,9.962,0.010,-4.347,-3.767,0.02,15.49,361,2020-06-02 10:25:00
3,2020-06-02 11:05:00,369,5.20,258.937,8.390,-0.357,-6.360,-1.258,0.03,20.04,374,2020-06-02 11:30:00
4,2020-06-02 11:35:00,375,5.18,284.889,7.608,-0.497,-6.297,1.677,0.03,9.21,380,2020-06-02 12:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...
143,2020-08-25 00:10:00,23539,6.20,296.891,16.946,0.578,-3.435,1.737,0.04,13.08,23544,2020-08-25 00:35:00
144,2020-08-25 06:00:00,23609,6.42,275.297,10.694,1.407,-3.890,0.363,0.10,16.33,23614,2020-08-25 06:25:00
145,2020-08-25 07:05:00,23622,6.35,304.304,9.199,-0.730,-4.332,2.957,0.03,29.44,23627,2020-08-25 07:30:00
146,2020-08-25 16:25:00,23734,6.13,311.393,10.028,-3.442,-2.453,2.073,0.10,22.38,23739,2020-08-25 16:50:00


,Start_str,Start_ind,Mgs#_avg,Clang_avg,Rmag_avg,Bx_avg(nT),By_avg(nT),Bz_avg(nT),Bdeviat,Dtheta,End_ind,End_str
0,2021-06-01 00:50:00,10,5.43,244.331,21.428,0.613,-2.403,-1.138,0.07,13.79,15,2021-06-01 01:15:00
1,2021-06-01 01:20:00,16,5.42,245.989,21.123,-0.053,-2.787,-1.243,0.05,9.68,21,2021-06-01 01:45:00
2,2021-06-01 04:10:00,50,5.05,256.041,19.220,1.367,-2.745,-0.673,0.04,22.65,55,2021-06-01 04:35:00
3,2021-06-01 08:40:00,104,6.95,245.306,15.479,-0.295,-1.075,-0.493,0.06,14.51,109,2021-06-01 09:05:00
4,2021-06-01 09:10:00,110,6.92,242.842,14.998,-0.210,-1.133,-0.583,0.08,10.50,115,2021-06-01 09:35:00
...,...,...,...,...,...,...,...,...,...,...,...,...
194,2021-08-24 03:15:00,22445,5.43,283.072,22.196,2.280,-4.657,1.077,0.08,16.00,22450,2021-08-24 03:40:00
195,2021-08-24 10:20:00,22530,5.37,248.279,17.246,3.765,-3.012,-1.208,0.05,6.67,22535,2021-08-24 10:45:00
196,2021-08-24 16:40:00,22606,5.78,296.219,10.594,0.145,-5.103,2.523,0.09,27.65,22611,2021-08-24 17:05:00
197,2021-08-24 17:10:00,22612,5.80,272.861,9.927,0.713,-6.123,0.295,0.06,20.93,22617,2021-08-24 17:35:00


Displaying Nightside Yr 1-7 General Stable Lists


### Section 1.1:
* Creating 10min Sublist from each 30min interval in the Yearly Stable IMF (7to24Re) List
* (9-9-2021) As discussed with Rick, the suggestion for the last 10 or 15min of the stable 30min interval was in regards to accounting for any OMNI data timing error. Additionally, since theres a 2500pts difference between using 10min and 15min, the difference shouldn't matter since I'm extracting the median(to rid mmyself of outliers) and other stat parameters of the desired curlometer parameters
    * **I'm going with collecting the last 10min for each stable 30min interval**

In [4]:
###Extracting 10min sublists from main Stable IMF events###############################
#print(df_NightsoBz_yr1.dtypes) #displaying column name and data type
print(df_DaysoBz_yr1.dtypes) #displaying column name and data type
def namestr(obj, namespace):
## (as defined by the a stackoverflowuser) returns name of object as a string list
    return [name for name in namespace if namespace[name] is obj]


def convert_strlist_to_unix(df_yr1,df_yr2,df_yr3, df_yr4, df_yr5):
    """Converting string end-date arrays from Yearly IMF DFs back to unix time"""
    #extracting string end-date arrays from Yearly Dataframes
    endstr_yr1 = df_yr1['End_str'].to_numpy(copy=True)
    endstr_yr2 = df_yr2['End_str'].to_numpy(copy=True)
    endstr_yr3 = df_yr3['End_str'].to_numpy(copy=True)
    endstr_yr4 = df_yr4['End_str'].to_numpy(copy=True)
    endstr_yr5 = df_yr5['End_str'].to_numpy(copy=True)
    #converting end-date string arrays to unix time
    endyr1_unix = time_double(endstr_yr1)
    endyr2_unix = time_double(endstr_yr2)
    endyr3_unix = time_double(endstr_yr3)
    endyr4_unix = time_double(endstr_yr4)
    endyr5_unix = time_double(endstr_yr5)
    return(endstr_yr1, endstr_yr2, endstr_yr3, endstr_yr4, endstr_yr5,
          endyr1_unix, endyr2_unix, endyr3_unix, endyr4_unix, endyr5_unix);

def extract_clang(df_yr1,df_yr2,df_yr3, df_yr4, df_yr5):
    """Extract copies of avg IMF Clock Angle arrays from Yearly DFs"""
    #extracting avg Clang arrays from Yearly Dataframes
    clang_yr1 = df_yr1['Clang_avg'].to_numpy(copy=True)
    clang_yr2 = df_yr2['Clang_avg'].to_numpy(copy=True)
    clang_yr3 = df_yr3['Clang_avg'].to_numpy(copy=True)
    clang_yr4 = df_yr4['Clang_avg'].to_numpy(copy=True)
    clang_yr5 = df_yr5['Clang_avg'].to_numpy(copy=True)
    return(clang_yr1, clang_yr2, clang_yr3, clang_yr4, clang_yr5)

def extract_bx(df_yr1,df_yr2,df_yr3, df_yr4, df_yr5):
    """Extract copies of avg IMF Bx arrays from Yearly DFs"""
    #extracting avg Clang arrays from Yearly Dataframes
    bx_yr1 = df_yr1['Bx_avg(nT)'].to_numpy(copy=True)
    bx_yr2 = df_yr2['Bx_avg(nT)'].to_numpy(copy=True)
    bx_yr3 = df_yr3['Bx_avg(nT)'].to_numpy(copy=True)
    bx_yr4 = df_yr4['Bx_avg(nT)'].to_numpy(copy=True)
    bx_yr5 = df_yr5['Bx_avg(nT)'].to_numpy(copy=True)
    return(bx_yr1, bx_yr2, bx_yr3, bx_yr4, bx_yr5)

def extract_by(df_yr1,df_yr2,df_yr3, df_yr4, df_yr5):
    """Extract copies of avg IMF By arrays from Yearly DFs"""
    #extracting avg Clang arrays from Yearly Dataframes
    by_yr1 = df_yr1['By_avg(nT)'].to_numpy(copy=True)
    by_yr2 = df_yr2['By_avg(nT)'].to_numpy(copy=True)
    by_yr3 = df_yr3['By_avg(nT)'].to_numpy(copy=True)
    by_yr4 = df_yr4['By_avg(nT)'].to_numpy(copy=True)
    by_yr5 = df_yr5['By_avg(nT)'].to_numpy(copy=True)
    return(by_yr1, by_yr2, by_yr3, by_yr4, by_yr5)

def extract_bz(df_yr1,df_yr2,df_yr3, df_yr4, df_yr5):
    """Extract copies of avg IMF Bz arrays from Yearly DFs"""
    #extracting avg Clang arrays from Yearly Dataframes
    bz_yr1 = df_yr1['Bz_avg(nT)'].to_numpy(copy=True)
    bz_yr2 = df_yr2['Bz_avg(nT)'].to_numpy(copy=True)
    bz_yr3 = df_yr3['Bz_avg(nT)'].to_numpy(copy=True)
    bz_yr4 = df_yr4['Bz_avg(nT)'].to_numpy(copy=True)
    bz_yr5 = df_yr5['Bz_avg(nT)'].to_numpy(copy=True)
    return(bz_yr1, bz_yr2, bz_yr3, bz_yr4, bz_yr5)

#fSum = 0
#for i in range(len(endstr_test)):
#    fSum = fSum + (end_unix[i] - start_unix[i])
#print('Avg diff between 30min unix time intervals was %0.3f'%(fSum/len(endstr_test)))

def unix_interval(unixEND):
##Copmuting/displaying startdate using the unix-time array for the end dates
    #The number of digits between the start and endpoint for a 30minute interval is 1500pts
    #15minute interval is actually 900pts and 600pts for 10min
    for i in range(len(unixEND)):
        endSTR = time_string(unixEND[i])
        startSTR = time_string(unixEND[i] - 600)
        print('\nWith the enddate, %0.19s, the 10minute start should be %0.19s'%(endSTR, startSTR))
    return;

'''
#unix_interval(end_unix)
#print(startstr_test[0:10])
print(list(endstr_test[0])) #displaying all characters for a string
print('Yes:', endstr_test[0][10]) #displaying specific character in string
love = endstr_test[0].replace(' ', '/') #adding slash mark to string date for trange input syntax
loveunix = time_double(endstr_test[0])
lovestr = endstr_test[0]
lovestr2 = time_string(loveunix)
print('As a string = %s, \nas a unix time float number= %0.1f, \nand reconverted back to string %s'
      %(lovestr, loveunix, lovestr2))
''';


def last10min_Stable(unixEND, strEND, afClang, afBx, afBy, afBz):
    """Creating two dataframes of string dates for the last 10min of Yearly Stable IMF intervals
    The 2nd dataframe will add '/' to string date to abide by trange input syntax"""
    #creating empty dataframes with column name
    df_last10 = pd.DataFrame(columns = ['Start_str','End_str', 'Clang_avg', 
                                        'Bx_avg(nT)', 'By_avg(nT)', 'Bz_avg(nT)']) #last10m with no slash mark
    df_10slash = pd.DataFrame(columns = ['Start_trang', 'End_trang', 'Clang_avg', 
                                         'Bx_avg(nT)', 'By_avg(nT)', 'Bz_avg(nT)']) #last10m with a '/' mark
    #print(df_last10)
        #print(df_Bound) #should display message if empty
    #Appending rows to empty dataframes using a loop
    for i in range(len(unixEND)):
        #Computing startdate using the unix-time array for the end dates
        #The number of digits between the start and endpoint for a 10min interval is 600pts
        endSTR = time_string(unixEND[i])
        startSTR = time_string(unixEND[i] - 600)
        clang = afClang[i] #defining variable for IMF Clang orientation to further differentiate DFs
        bx_val = afBx[i]
        by_val = afBy[i]
        bz_val = afBz[i]
        #creating other variables that add slash mark for trange input syntax
        endTRAN = endSTR.replace(' ', '/') #replace any ' ' marks with a '/' mark
        startTRAN = startSTR.replace(' ', '/')
        df_last10 = df_last10.append({'Start_str': '%0.19s'%(startSTR), 'End_str': '%0.19s'%(endSTR), 
                           'Clang_avg': round(clang,3), 'Bx_avg(nT)': round(bx_val,3),
                            'By_avg(nT)': round(by_val,3), 'Bz_avg(nT)': round(bz_val,3)}, ignore_index=True)
        df_10slash = df_10slash.append({'Start_trang': '%0.19s'%(startTRAN), 'End_trang': '%0.19s'%(endTRAN),
                           'Clang_avg': round(clang,3), 'Bx_avg(nT)': round(bx_val,3),
                           'By_avg(nT)': round(by_val,3), 'Bz_avg(nT)': round(bz_val,3)}, ignore_index=True)
    #print(df_last10['Start_str']) 
    #display(df_last10.iloc[0:3])
    #display(df_10slash)
    #print((type(df_last10['Start_str'][0])))
        #to display full dataframe in full text format do print(df.to_string())
        #use df.iloc[[row], [col]] to index into dataframe
    compare_10min_30min(strEND, df_last10)
        ##confirmed that the arrays are the same
    return(df_10slash, df_last10);

def compare_10min_30min(full30, df_last10m):
#Compares dataframes columns for End-dates between 30min and 10min dataframe. They should be the same
    last10 = df_last10m['End_str'].to_numpy(copy=True)
    print('Full30 and Last10 minute Dataframes end-date columns are the same: ', (last10 == full30).all())
    return;


def compare_imp_df(df_og, df_imp):
# Compares the original and reimported dataframes to ensure that they hold the same element values
# Also takes into account df.equals() error with the imported DF changing the column types from the original
    #Checks for column array equality. The first accounts for dtype while the 2nd ignores it
    print((df_og == df_imp).all()) 
    print('\nOG and imported DF are the same:', all(df_og == df_imp)) 
    df_imp2 = df_imp.copy(deep=True)
    print('Imported and copy of imported DF are the same:', df_imp2.equals(df_imp))
    print('Using df.equals,OG and reimported dataframes are the same:', df_og.equals(df_imp))
    #Displaying dataframes and columns dtype if needed
    #print(df_imp_30Filt_yr1.dtypes)
    display(df_og[:3])
    display(df_imp[:3])
    return;

def compare_clang(df_og, df_NOsl_10m):
    """Compare ClockAng column arrays between OG and Last10m DFs (should be the same)"""
    print('\nComparing column arrays between: ',namestr(df_og, globals()), namestr(df_NOsl_10m, globals()) )
    clang_noslash = df_NOsl_10m['Clang_avg'].to_numpy(copy=True)
    clang_og = df_og['Clang_avg'].to_numpy(copy=True)
    print('OG and Last10 minute Dataframes clang_avg columns are the same: ', (clang_noslash == clang_og).all())
    #print('CLang columns are the same: ',  np.allclose(clang_noslash,clang_og, equal_nan=True))
    bx_nosl = df_NOsl_10m['Bx_avg(nT)'].to_numpy(copy=True)
    by_nosl = df_NOsl_10m['By_avg(nT)'].to_numpy(copy=True)
    bz_nosl = df_NOsl_10m['Bz_avg(nT)'].to_numpy(copy=True)
    bx_og = df_og['Bx_avg(nT)'].to_numpy(copy=True)
    by_og = df_og['By_avg(nT)'].to_numpy(copy=True)
    bz_og = df_og['Bz_avg(nT)'].to_numpy(copy=True)
    print('\tOG and Last10 minute DFs Bx, By, and Bz columns are the same: ', (bx_nosl == bx_og).all(),
         (by_nosl == by_og).all(), (bz_nosl == bz_og).all())
    #print('\tBcomp columns are the same: ', np.allclose(bx_nosl,bx_og, equal_nan=True), np.allclose(by_nosl,by_og, equal_nan=True), 
     #      np.allclose(bz_nosl,bz_og, equal_nan=True))
    #print('\tAvg ratio and abs difference of OG and Last10m Clang_avg columns:', 
    #      np.average(clang_og/clang_noslash) ,np.average(np.abs(clang_og - clang_noslash)))
    #Confirmed: the same for negBy
    return;



#################### Main Calling Function####################
def pd_extract_last10(df_yr1,df_yr2,df_yr3, df_yr4, df_yr5):
    """From the 30minute intervals of the inputted <ClockAng> Stable IMF Lists, extract out the last 10min 
    of the timeframes for curlometer data calculations"""
    #Extracting string date-lists and unix time arrays from Yrly Stable IMF DFs
    [endstr_yr1, endstr_yr2, endstr_yr3, endstr_yr4, endstr_yr5, endyr1_unix, endyr2_unix, endyr3_unix, 
     endyr4_unix, endyr5_unix] = convert_strlist_to_unix(df_yr1, df_yr2, df_yr3, df_yr4, df_yr5)
    
    #Extracting copies of avg ClockAng and Bx arrays from Yrly DFs
    [clang_yr1, clang_yr2, clang_yr3, clang_yr4, clang_yr5]= extract_clang(df_yr1,df_yr2,df_yr3,df_yr4, df_yr5)
    [bx_yr1, bx_yr2, bx_yr3, bx_yr4, bx_yr5]= extract_bx(df_yr1,df_yr2,df_yr3,df_yr4, df_yr5)
    [by_yr1, by_yr2, by_yr3, by_yr4, by_yr5]= extract_by(df_yr1,df_yr2,df_yr3,df_yr4, df_yr5)
    [bz_yr1, bz_yr2, bz_yr3, bz_yr4, bz_yr5]= extract_bz(df_yr1,df_yr2,df_yr3,df_yr4, df_yr5)

    
    # Extracting Yr1's last 10min from <ClockAng> 30min list
    [df_10m_yr1, df_NOSL10m_yr1] = last10min_Stable(endyr1_unix, endstr_yr1, clang_yr1, bx_yr1, by_yr1, bz_yr1)
       #Exporting yr1 last10min Filtered Data and comparing the exported/OG dataframe
    #df_NOSL10m_yr1.to_csv('pdFile_DayNOSL_Yr1_last10m_trange_posBy.txt', sep = '\t', index = False)
    #df_imp_10min_yr1 = pd.read_csv('pdFile_DayNOSL_Yr1_last10m_trange_posBy.txt', sep = '\t')
    #compare_imp_df(df_NOSL10m_yr1, df_imp_10min_yr1)

    # Extracting Yr2's last 10min from <ClockAng> 30min list
    [df_10m_yr2, df_NOSL10m_yr2] = last10min_Stable(endyr2_unix, endstr_yr2, clang_yr2, bx_yr2,by_yr2, bz_yr2)
        #Exporting yr2 last10min Filtered Data and comparing the exported/OG dataframe
    #df_NOSL10m_yr2.to_csv('pdFile_DayNOSL_Yr2_last10m_trange_posBy.txt', sep = '\t', index = False)
    #df_imp_10min_yr2 = pd.read_csv('pdFile_DayNOSL_Yr2_last10m_trange_posBy.txt', sep = '\t')
    #compare_imp_df(df_NOSL10m_yr2, df_imp_10min_yr2)

    # Extracting Yr3's last 10min from <ClockAng> 30min list
    [df_10m_yr3, df_NOSL10m_yr3] = last10min_Stable(endyr3_unix, endstr_yr3, clang_yr3, bx_yr3,by_yr3, bz_yr3)
        #Exporting yr3 last10min Filtered Data and comparing the exported/OG dataframe
    #df_NOSL10m_yr3.to_csv('pdFile_DayNOSL_Yr3_last10m_trange_posBy.txt', sep = '\t', index = False)
    #df_imp_10min_yr3 = pd.read_csv('pdFile_DayNOSL_Yr3_last10m_trange_posBy.txt', sep = '\t')
    #compare_imp_df(df_NOSL10m_yr3, df_imp_10min_yr3)

    # Extracting Yr4's last 10min from <ClockAng> 30min list
    [df_10m_yr4, df_NOSL10m_yr4] = last10min_Stable(endyr4_unix, endstr_yr4, clang_yr4, bx_yr4,by_yr4, bz_yr4)
        #Exporting yr4 last10min Filtered Data and comparing the exported/OG dataframe
    #df_NOSL10m_yr4.to_csv('pdFile_DayNOSL_Yr4_last10m_trange_posBy.txt', sep = '\t', index = False)
    #df_imp_10min_yr4 = pd.read_csv('pdFile_DayNOSL_Yr4_last10m_trange_posBy.txt', sep = '\t')
    #compare_imp_df(df_NOSL10m_yr4, df_imp_10min_yr4)

    # Extracting Yr5's last 10min from <ClockAng> 30min list
    [df_10m_yr5, df_NOSL10m_yr5] = last10min_Stable(endyr5_unix, endstr_yr5, clang_yr5, bx_yr5,by_yr5, bz_yr5)
        #Exporting yr5 last10min Filtered Data and comparing the exported/OG dataframe
    #df_NOSL10m_yr5.to_csv('pdFile_DayNOSL_Yr5_last10m_trange_posBy_noGAP_incr.txt', sep = '\t', index = False)
    #df_imp_10min_yr5 = pd.read_csv('pdFile_DayNOSL_Yr5_last10m_trange_posBy_noGAP_incr.txt', sep = '\t')
    #compare_imp_df(df_NOSL10m_yr5, df_imp_10min_yr5)
    
    return(df_NOSL10m_yr1, df_NOSL10m_yr2, df_NOSL10m_yr3, df_NOSL10m_yr4, df_NOSL10m_yr5);

#Extracting No-slash last10 dataframes and displaying them
'''
print('Extracting SoBz last 10min sublists from Yrly DFs, Theta:[135, 225]')
[df_sBz_noslash10m_yr1, df_sBz_noslash10m_yr2, df_sBz_noslash10m_yr3, df_sBz_noslash10m_yr4, 
                     df_sBz_noslash10m_yr5] = pd_extract_last10(df_DaysoBz_yr1,df_DaysoBz_yr2,
                                    df_DaysoBz_yr3,df_DaysoBz_yr4, df_DaysoBz_yr5)

#Comparing clang_avg columns
compare_clang(df_DaysoBz_yr1, df_sBz_noslash10m_yr1)
compare_clang(df_DaysoBz_yr2, df_sBz_noslash10m_yr2)
compare_clang(df_DaysoBz_yr3, df_sBz_noslash10m_yr3)
compare_clang(df_DaysoBz_yr4, df_sBz_noslash10m_yr4)
compare_clang(df_DaysoBz_yr5, df_sBz_noslash10m_yr5)
#Showcase Yr1to5 soBz NoSlash10m DFs
#show1to5DF(df_sBz_noslash10m_yr1, df_sBz_noslash10m_yr2, df_sBz_noslash10m_yr3, df_sBz_noslash10m_yr4, df_sBz_noslash10m_yr5)
''';

'''
print('n\Extracting NegBy last 10min sublists from Yrly DFs, Theta:[225, 315]')
[df_nBy_noslash10m_yr1, df_nBy_noslash10m_yr2, df_nBy_noslash10m_yr3, df_nBy_noslash10m_yr4, 
                     df_nBy_noslash10m_yr5] = pd_extract_last10(df_DaynegBy_yr1, df_DaynegBy_yr2, 
                                                df_DaynegBy_yr3, df_DaynegBy_yr4, df_DaynegBy_yr5)

#Comparing clang_avg columns
compare_clang(df_DaynegBy_yr1, df_nBy_noslash10m_yr1)
compare_clang(df_DaynegBy_yr2, df_nBy_noslash10m_yr2)
compare_clang(df_DaynegBy_yr3, df_nBy_noslash10m_yr3)
compare_clang(df_DaynegBy_yr4, df_nBy_noslash10m_yr4)
compare_clang(df_DaynegBy_yr5, df_nBy_noslash10m_yr5)
#Showcase Yr1to5 negBy NoSlash10m DFs
#show1to5DF(df_nBy_noslash10m_yr1, df_nBy_noslash10m_yr2, df_nBy_noslash10m_yr3, df_nBy_noslash10m_yr4, df_nBy_noslash10m_yr5)
''';

#'''
print('n\Extracting PosBy last 10min sublists from Yrly DFs, Theta:[45, 135]')
[df_pBy_noslash10m_yr1, df_pBy_noslash10m_yr2, df_pBy_noslash10m_yr3, df_pBy_noslash10m_yr4, 
                     df_pBy_noslash10m_yr5] = pd_extract_last10(df_DayposBy_yr1, df_DayposBy_yr2, 
                                df_DayposBy_yr3, df_DayposBy_yr4, df_DayposBy_yr5)

#Comparing clang_avg columns
compare_clang(df_DayposBy_yr1, df_pBy_noslash10m_yr1)
compare_clang(df_DayposBy_yr2, df_pBy_noslash10m_yr2)
compare_clang(df_DayposBy_yr3, df_pBy_noslash10m_yr3)
compare_clang(df_DayposBy_yr4, df_pBy_noslash10m_yr4)
compare_clang(df_DayposBy_yr5, df_pBy_noslash10m_yr5)
#Showcase Yr1to5 posBy NoSlash10m DFs
#show1to5DF(df_pBy_noslash10m_yr1, df_pBy_noslash10m_yr2, df_pBy_noslash10m_yr3, df_pBy_noslash10m_yr4, df_pBy_noslash10m_yr5)
#''';


'''
print('n\Extracting NorBz last 10min sublists from Yrly DFs, Theta:[315, 45]')
[df_noBz_noslash10m_yr1, df_noBz_noslash10m_yr2, df_noBz_noslash10m_yr3, df_noBz_noslash10m_yr4, 
                     df_noBz_noslash10m_yr5] = pd_extract_last10(df_DaynorBz_yr1, df_DaynorBz_yr2, 
                                 df_DaynorBz_yr3, df_DaynorBz_yr4, df_DaynorBz_yr5)

#Comparing clang_avg columns
compare_clang(df_DaynorBz_yr1, df_noBz_noslash10m_yr1)
compare_clang(df_DaynorBz_yr2, df_noBz_noslash10m_yr2)
compare_clang(df_DaynorBz_yr3, df_noBz_noslash10m_yr3)
compare_clang(df_DaynorBz_yr4, df_noBz_noslash10m_yr4)
compare_clang(df_DaynorBz_yr5, df_noBz_noslash10m_yr5)
#Showcase Yr1to5 norBz NoSlash10m DFs
#show1to5DF(df_noBz_noslash10m_yr1, df_noBz_noslash10m_yr2, df_noBz_noslash10m_yr3, df_noBz_noslash10m_yr4, df_noBz_noslash10m_yr5)
''';


'''
print('Extracting General NonClangSort last 10min sublists from Yrly DFs')
[df_gen_noslash10m_yr1, df_gen_noslash10m_yr2, df_gen_noslash10m_yr3, df_gen_noslash10m_yr4, 
                     df_gen_noslash10m_yr5] = pd_extract_last10(df_Daystable_yr1,df_Daystable_yr2,
                                df_Daystable_yr3,df_Daystable_yr4, df_Daystable_yr5)

#Comparing clang_avg columns
compare_clang(df_Daystable_yr1, df_gen_noslash10m_yr1)
compare_clang(df_Daystable_yr2, df_gen_noslash10m_yr2)
compare_clang(df_Daystable_yr3, df_gen_noslash10m_yr3)
compare_clang(df_Daystable_yr4, df_gen_noslash10m_yr4)
compare_clang(df_Daystable_yr5, df_gen_noslash10m_yr5)
#Showcase Yr1to5 General NoSlash10m DFs
#show1to5DF(df_gen_noslash10m_yr1, df_gen_noslash10m_yr2, df_gen_noslash10m_yr3, df_gen_noslash10m_yr4, df_gen_noslash10m_yr5)
''';


Start_str      object
Start_ind       int64
Mgs#_avg      float64
Clang_avg     float64
Rmag_avg      float64
Bx_avg(nT)    float64
By_avg(nT)    float64
Bz_avg(nT)    float64
Bdeviat       float64
Dtheta        float64
End_ind         int64
End_str        object
dtype: object
n\Extracting PosBy last 10min sublists from Yrly DFs, Theta:[45, 135]
Full30 and Last10 minute Dataframes end-date columns are the same:  True
Full30 and Last10 minute Dataframes end-date columns are the same:  True
Full30 and Last10 minute Dataframes end-date columns are the same:  True
Full30 and Last10 minute Dataframes end-date columns are the same:  True
Full30 and Last10 minute Dataframes end-date columns are the same:  True

Comparing column arrays between:  ['df_DayposBy_yr1'] ['df_pBy_noslash10m_yr1']
OG and Last10 minute Dataframes clang_avg columns are the same:  True
	OG and Last10 minute DFs Bx, By, and Bz columns are the same:  True True True

Comparing column arrays between:  ['df_DayposBy_yr2'] ['

* (5-24-2022) Creating Last 10min sublist from 2020-21's Night/Day Stable 30min IMF List

In [5]:
def single_convert_strlist_to_unix(df_yr1):
    """Converting string end-date arrays from Yearly IMF DFs back to unix time"""
    #extracting string end-date arrays from Yearly Dataframes
    endstr_yr1 = df_yr1['End_str'].to_numpy(copy=True)
    #converting end-date string arrays to unix time
    endyr1_unix = time_double(endstr_yr1)
    return(endstr_yr1, endyr1_unix);

def single_extract_clang(df_yr1):
    """Extract copies of avg IMF Clock Angle arrays from Yearly DFs"""
    #extracting avg Clang arrays from Yearly Dataframes
    clang_yr1 = df_yr1['Clang_avg'].to_numpy(copy=True)
    return(clang_yr1)

def single_extract_bx(df_yr1):
    """Extract copies of avg IMF Bx arrays from Yearly DFs"""
    #extracting avg Clang arrays from Yearly Dataframes
    bx_yr1 = df_yr1['Bx_avg(nT)'].to_numpy(copy=True)
    return(bx_yr1)

def single_extract_by(df_yr1):
    """Extract copies of avg IMF By arrays from Yearly DFs"""
    #extracting avg Clang arrays from Yearly Dataframes
    by_yr1 = df_yr1['By_avg(nT)'].to_numpy(copy=True)
    return(by_yr1)

def single_extract_bz(df_yr1):
    """Extract copies of avg IMF Bz arrays from Yearly DFs"""
    #extracting avg Clang arrays from Yearly Dataframes
    bz_yr1 = df_yr1['Bz_avg(nT)'].to_numpy(copy=True)
    return(bz_yr1)

def single_extract_IMFList_params(df_yr1):
    """From YrX IMF DF extract array copies of: end-date string and unix time; avg IMF Clock Angle,
    and avg IMF Bcomps"""
    #extracting string end-date arrays from Yearly Dataframes
    endstr_yr1 = df_yr1['End_str'].to_numpy(copy=True)
    #converting end-date string arrays to unix time
    endyr1_unix = time_double(endstr_yr1)
    #extracting avg Clang arrays from Yearly Dataframes
    clang_yr1 = df_yr1['Clang_avg'].to_numpy(copy=True)
    #extracting avg IMF Bx,By, and Bz arrays 
    bx_yr1 = df_yr1['Bx_avg(nT)'].to_numpy(copy=True)
    by_yr1 = df_yr1['By_avg(nT)'].to_numpy(copy=True)
    bz_yr1 = df_yr1['Bz_avg(nT)'].to_numpy(copy=True)
    return(endstr_yr1, endyr1_unix, clang_yr1, bx_yr1, by_yr1, bz_yr1);

def compare_clang_V2(df_og, df_NOsl_10m):
    """Compare ClockAng column arrays between OG and Last10m DFs (should be the same)"""
    clang_noslash = df_NOsl_10m['Clang_avg'].to_numpy(copy=True)
    clang_og = df_og['Clang_avg'].to_numpy(copy=True)
    print(len(clang_noslash), len(clang_og))
    #print('OG and Last10 minute Dataframes clang_avg columns are the same: ', (clang_noslash == clang_og).all())
    #print('CLang columns are the same: ',  np.allclose(clang_noslash,clang_og, equal_nan=True))
    bx_nosl = df_NOsl_10m['Bx_avg(nT)'].to_numpy(copy=True)
    by_nosl = df_NOsl_10m['By_avg(nT)'].to_numpy(copy=True)
    bz_nosl = df_NOsl_10m['Bz_avg(nT)'].to_numpy(copy=True)
    bx_og = df_og['Bx_avg(nT)'].to_numpy(copy=True)
    by_og = df_og['By_avg(nT)'].to_numpy(copy=True)
    bz_og = df_og['Bz_avg(nT)'].to_numpy(copy=True)
    #print('\tOG and Last10 minute DFs Bx, By, and Bz columns are the same: ', (bx_nosl == bx_og).all(),
     #    (by_nosl == by_og).all(), (bz_nosl == bz_og).all())
    #print('\tBcomp columns are the same: ', np.allclose(bx_nosl,bx_og, equal_nan=True), np.allclose(by_nosl,by_og, equal_nan=True), 
     #      np.allclose(bz_nosl,bz_og, equal_nan=True))
    #print('\tAvg ratio and abs difference of OG and Last10m Clang_avg columns:', 
    #      np.average(clang_og/clang_noslash) ,np.average(np.abs(clang_og - clang_noslash)))
    #Confirmed: the same for negBy

def single_pd_extract_last10(df_yrX):
    """From the 30minute intervals of the inputted YrX <ClockAng> Stable IMF List, extract out the last 10min 
    of the timeframes for curlometer data calculations"""
    print('\nCreating Last10min Sublist from: ',namestr(df_yrX, globals()))
    #Extracting string date-lists and unix time arrays from YrX Stable IMF DFs
    #[endstr_yr1, endyr1_unix] = convert_strlist_to_unix(df_yrX)
    #Extracting copies of avg ClockAng and IMF Bcomp arrays from YrX DFs
    #clang_yr1 = extract_clang(df_yrX)
    #bx_yr1 = extract_bx(df_yrX)
    #by_yr1 = extract_by(df_yrX)
    #bz_yr1 = extract_bz(df_yrX)
    #extract string date-list, unix time array, avg ClockAng, and avg IMF Bcomp arrays from YrX DF
    [endstr_yr1, endyr1_unix, clang_yr1, bx_yr1, by_yr1, bz_yr1] = single_extract_IMFList_params(df_yrX)    
    # Extracting Yr1's last 10min from <ClockAng> 30min list
    [df_SL10m_yr1, df_NOSL10m_yr1] = last10min_Stable(endyr1_unix, endstr_yr1, clang_yr1, bx_yr1, by_yr1, bz_yr1)
       #Exporting yr1 last10min Filtered Data and comparing the exported/OG dataframe
    #df_NOSL10m_yr1.to_csv('pdFile_NightNOSL_Yr7_last10m_trange_general.txt', sep = '\t', index = False)
    #df_imp_10min_yr1 = pd.read_csv('pdFile_NightNOSL_Yr7_last10m_trange_general.txt', sep = '\t')
    #compare_imp_df(df_NOSL10m_yr1, df_imp_10min_yr1)
    return(df_NOSL10m_yr1);

#----------------------Fundamental Functions Above/Main Calling Below----------
##---soBz--Extracting No-slash last10min dataframes 
'''
print('Extracting SoBz last 10min sublists from Yr6(2020-21) DF, Theta:[135, 225)')
#df_sBz_noslash10m_yr6 = single_pd_extract_last10(df_DaysoBz_yr6)
    #Compare clang_avg columns and display Yr6 soBz NoSlash10m DF
#compare_clang(df_DaysoBz_yr6, df_sBz_noslash10m_yr6)

#df_sBz_noslash10m_yr6 = single_pd_extract_last10(df_NightsoBz_yr6)
#compare_clang(df_NightsoBz_yr6, df_sBz_noslash10m_yr6)

#df_sBz_noslash10m_yr7 = single_pd_extract_last10(df_NightsoBz_yr7)
#compare_clang(df_NightsoBz_yr7, df_sBz_noslash10m_yr7)
#display(df_sBz_noslash10m_yr7)
''';


##---norBz--Extracting No-slash last10min dataframes 
'''
print('Extracting norBz last 10min sublists from Yr6(2020-21) DF, Theta:[315, 45)')
#df_nBz_noslash10m_yr6 = single_pd_extract_last10(df_DaynorBz_yr6)
    #Compare clang_avg columns and display Yr6 norBz NoSlash10m DF
#compare_clang(df_DaynorBz_yr6, df_nBz_noslash10m_yr6)

#df_nBz_noslash10m_yr6 = single_pd_extract_last10(df_NightnorBz_yr6)
#compare_clang(df_NightnorBz_yr6, df_nBz_noslash10m_yr6)


df_nBz_noslash10m_yr7 = single_pd_extract_last10(df_NightnorBz_yr7)
compare_clang(df_NightnorBz_yr7, df_nBz_noslash10m_yr7)
#display(df_nBz_noslash10m_yr7)
''';

##---posBy--Extracting No-slash last10min dataframes 
'''
print('Extracting posBy last 10min sublists from Yr6(2020-21) DF, Theta:[45, 135)')
#df_pBy_noslash10m_yr6 = single_pd_extract_last10(df_DayposBy_yr6)
    #Compare clang_avg columns and display Yr6 posBy NoSlash10m DF
#compare_clang(df_DayposBy_yr6, df_pBy_noslash10m_yr6)

#df_pBy_noslash10m_yr6 = single_pd_extract_last10(df_NightposBy_yr6)
#compare_clang(df_NightposBy_yr6, df_pBy_noslash10m_yr6)

df_pBy_noslash10m_yr7 = single_pd_extract_last10(df_NightposBy_yr7)
compare_clang(df_NightposBy_yr7, df_pBy_noslash10m_yr7)
display(df_pBy_noslash10m_yr7)
''';

##---negBy--Extracting No-slash last10min dataframes 
'''
print('Extracting negBy last 10min sublists from Yr6(2020-21) DF, Theta:[225, 315)')
#df_nBy_noslash10m_yr6 = single_pd_extract_last10(df_DaynegBy_yr6)
    #Compare clang_avg columns and display Yr6 negBy NoSlash10m DF
#compare_clang(df_DaynegBy_yr6, df_nBy_noslash10m_yr6)

#df_nBy_noslash10m_yr6 = single_pd_extract_last10(df_NightnegBy_yr6)
#compare_clang(df_NightnegBy_yr6, df_nBy_noslash10m_yr6)

df_nBy_noslash10m_yr7 = single_pd_extract_last10(df_NightnegBy_yr7)
compare_clang(df_NightnegBy_yr7, df_nBy_noslash10m_yr7)
display(df_nBy_noslash10m_yr7)
''';

##---General--Extracting No-slash last10min dataframes 
#'''
print('Extracting General NonClangSort last 10min sublists from Yr6(2020-21) DF')
#df_gen_noslash10m_yr6 = single_pd_extract_last10(df_Daystable_yr6)
    #Compare clang_avg columns and display Yr6 posBy NoSlash10m DF
#compare_clang(df_Daystable_yr6, df_gen_noslash10m_yr6)

#df_gen_noslash10m_yr6 = single_pd_extract_last10(df_Nightstable_yr6)
#compare_clang(df_Nightstable_yr6, df_gen_noslash10m_yr6)

df_gen_noslash10m_yr7 = single_pd_extract_last10(df_Nightstable_yr7)
compare_clang(df_Nightstable_yr7, df_gen_noslash10m_yr7)
display(df_gen_noslash10m_yr7)
#''';

Extracting General NonClangSort last 10min sublists from Yr6(2020-21) DF

Creating Last10min Sublist from:  ['df_Nightstable_yr7']
Full30 and Last10 minute Dataframes end-date columns are the same:  True

Comparing column arrays between:  ['df_Nightstable_yr7'] ['df_gen_noslash10m_yr7']
OG and Last10 minute Dataframes clang_avg columns are the same:  True
	OG and Last10 minute DFs Bx, By, and Bz columns are the same:  True True True


,Start_str,End_str,Clang_avg,Bx_avg(nT),By_avg(nT),Bz_avg(nT)
0,2021-06-01 01:05:00,2021-06-01 01:15:00,244.331,0.613,-2.403,-1.138
1,2021-06-01 01:35:00,2021-06-01 01:45:00,245.989,-0.053,-2.787,-1.243
2,2021-06-01 04:25:00,2021-06-01 04:35:00,256.041,1.367,-2.745,-0.673
3,2021-06-01 08:55:00,2021-06-01 09:05:00,245.306,-0.295,-1.075,-0.493
4,2021-06-01 09:25:00,2021-06-01 09:35:00,242.842,-0.210,-1.133,-0.583
...,...,...,...,...,...,...
506,2021-08-28 13:30:00,2021-08-28 13:40:00,112.512,-1.648,7.038,-2.913
507,2021-08-28 14:00:00,2021-08-28 14:10:00,98.548,-1.555,7.318,-1.103
508,2021-08-28 14:30:00,2021-08-28 14:40:00,95.961,-1.477,7.087,-0.740
509,2021-08-28 15:00:00,2021-08-28 15:10:00,88.390,-1.380,6.535,0.192


* In observing start and endbdates in unix times. The number of digits between the start and endpoint for a 30minute interval is 1500pts. Meaning a 15minute interval *should* be about 750pts
    * Results computed from Avg-For-Loop algorithm, <br>`fSum = fSum + (end_unix[i] - start_unix[i])` and confirmed by `unix_interval()` function
    * **It's 900pts for a 15minute interval and 600pts for 10min** as confirmed with number adjusting
    
* Replacing a character within a string: https://stackoverflow.com/questions/12723751/replacing-instances-of-a-character-in-a-string


### Section 1.a (SPECIAL)
#### (9-26-2021)Creating/Exporting MEC position data DF that corresponds to whichever exported ClockAng CurrData DF
* ALthough not useful in my context since I need both start and end-dates. Here's a quickhand way of creating sublist from a list, with a known array of desired indices 
    * https://stackoverflow.com/questions/22412509/getting-a-sublist-of-a-python-list-with-the-given-indices

In [6]:
######### Acquiring corresponding dataframe for MEC position data #########

print(sorted(yr5_MECposdata))
#find index of current Bmag time(trim) element that corresponds to OG time array
             #   mgs_in =  np.argwhere(ogtime == timetrim[n])
#                    pos_avg = np.average(posmag[mgs_in[0,0]: mgs_in[0,0] +6]/R_e)


def get_MEC_df(df_last10_noslash, yrX_MECposdata):
    """Create dataframe for MEC posdata using start-dates from last10m ClockAng Stable IMF Event Lists"""
    #create empty dataframe with column names
    df_MEC = pd.DataFrame(columns = ['Unix_start', 'Start_str','MECrmag_avg(Re)','MECx_avg(Re)',
                                     'MECy_avg(Re)', 'MECz_avg(Re)','End_str'])
    R_e = 6371 #km, Earth radius; as indicated by NRL Plasma Formularly
    #Displaying/extracting copy of start-date column from last10min Stable IMF df
    (display(df_last10_noslash))
    last10_start = df_last10_noslash['Start_str'].to_numpy(copy=True)
    #Extracting MEC radial position(km) and time(unix time) arrays
    afMECtime = yrX_MECposdata['t']
    afMECx = yrX_MECposdata['x']
    afMECy = yrX_MECposdata['y']
    afMECz = yrX_MECposdata['z']
    afMEC_rmag = yrX_MECposdata['rmag']
    print('\nAnalyzing MEC data time frame:', time_string(afMECtime[1]),'->\t', time_string(afMECtime[-1:]))
    # Using loop to append rows to empty dataframe by finding index for corresponding MEC position data
    count = 0 #initializing empty variable
    achMECtime = (time_string(afMECtime))  #Convert MECtime to from unix to string dates 
    for i in range(len(last10_start)):
        # Finding index of MECtime that corresponds to current last10_start element
        ind = np.argwhere(time_double(last10_start[i]) == afMECtime)
        # Defining variables for start/end-date and position(Re units)
        startMEC = achMECtime[ind[0,0]]
        endMEC = achMECtime[ind[0,0]+2] #MECtime array has 5min time cadence
        rmag_avg = np.nanmean(afMEC_rmag[ind[0,0]: ind[0,0] +3]/R_e)
        x_avg = np.nanmean(afMECx[ind[0,0]: ind[0,0] +3]/R_e)
        y_avg = np.nanmean(afMECy[ind[0,0]: ind[0,0] +3]/R_e)
        z_avg = np.nanmean(afMECz[ind[0,0]: ind[0,0] +3]/R_e)
        #Appending rows to empty dataframes using a loop
        if ( time_double(last10_start[i])==afMECtime[ind[0,0]] ):
            count = count +1
        #print(startMEC[:19])
        #print('Element %d matches: '%(i), (time_double(last10_start[i])==afMECtime[ind[0,0]]))
        if i < 1 or i > (len(last10_start)-2):
            print('For element %d: '%(i), achMECtime[ind[0,0]: ind[0,0]+3], '\n MECx(km):', afMECx[ind[0,0]: ind[0,0] +3])
        df_MEC = df_MEC.append({'Unix_start': '%f'%(afMECtime[ind[0,0]]), 'Start_str': '%0.19s'%(startMEC),
                                'MECrmag_avg(Re)': rmag_avg, 'MECx_avg(Re)': x_avg, 'MECy_avg(Re)': y_avg,
                                'MECz_avg(Re)': z_avg, 'End_str': '%0.19s'%(endMEC)},ignore_index=True)
    print('\tWith a length of %d for last10_start, there were %d confirmed date matches with MECtime'%(len(last10_start), count))
    display(df_MEC)    
    return(df_MEC);


def compare_mec_start_end(df_MECpos, df_last10m):
#Compares columns for Start and End-dates between MEC and last10m dataframe. They should be the same
    end_last10 = df_last10m['End_str'].to_numpy(copy=True)
    MECend = df_MECpos['End_str'].to_numpy(copy=True)
    MECstart = df_MECpos['Start_str'].to_numpy(copy=True)
    start_last10 = df_last10m['Start_str'].to_numpy(copy=True)
    print('MEC and Last10 minute Dataframes end-date columns are the same: ', (end_last10 == MECend).all())
    print('MEC and Last10 minute Dataframes start-date columns are the same: ', (start_last10 == MECstart).all())
    return;

def checkin_MEC_dfs(df_NSL10_yr1, df_NSL10_yr2, df_NSL10_yr3, df_NSL10_yr4, df_NSL10_yr5):
    """Checking newly made MEC position dataframes by ensuring date column arrays match last10m Stable List. 
    Then compare the exported/imported dataframes"""
    print('Inputs which are used to create MEC DFs:\n', namestr(df_NSL10_yr1, globals()),
          namestr(df_NSL10_yr2, globals()),namestr(df_NSL10_yr3, globals()), '\n', 
          namestr(df_NSL10_yr4, globals()), namestr(df_NSL10_yr5, globals()))
    
    #Exporting yr1 MEC posdata for each last10min interval and comparing the exported/OG dataframe
    #df_MECpos_yr1 = get_MEC_df(df_NSL10_yr1, yr1_MECposdata)
    #compare_mec_start_end(df_MECpos_yr1, df_NSL10_yr1)
    #df_MECpos_yr1.to_csv('pdFile_MECcurmatch_last10m_norBzdom_Yr1.txt', sep = '\t', index = False)
    #df_imp_MECpos_yr1 = pd.read_csv('pdFile_MECcurmatch_last10m_norBzdom_Yr1.txt', sep = '\t')
    #compare_imp_df(df_MECpos_yr1, df_imp_MECpos_yr1)
    
    #Exporting yr2 MEC posdata for each last10min interval and comparing the exported/OG dataframe
    #df_MECpos_yr2 = get_MEC_df(df_NSL10_yr2, yr2_MECposdata)
    #compare_mec_start_end(df_MECpos_yr2, df_NSL10_yr2)
    #For yr2_currdata(and mecdata): dropping NoData row
    #df_mec_y2 = df_MECpos_yr2.drop([105]) #yr2 norBz drop
    #df_mec_y2 = df_mec_y2.reset_index(drop=True)
    '''
    For norBz yr2_currdata
    #df_curr = df_currdata.drop([105]) #yr2 norBz drop
    
    
    #For posBy yr2_currdata(and mecdata): dropping NoData row
    #df_mec_y2 = df_MECpos_yr2.drop([481, 482, 483, 484, 485, 486, 487, 488, 489, 490])# yr2 posBy drop
    
    #For negBy yr2_currdata(and mec data): dropping NoData row
    #df_mec_y2 = df_MECpos_yr2.drop([234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244])# yr2 negBy drop
    #df_mec_y2 = df_mec_y2.reset_index(drop=True)
    
    #matching to yr2 soBz current df
    #df_mec2_yr2 = df_MECpos_yr2.drop([189, 190, 191, 192, 193, 194, 195]) #dropping NoData row for Yr2
    #df_mec2_yr2 = df_mec2_yr2.reset_index(drop = True) #reset index# column for new DF
    #print('\n New drop DF')
    #display(df_mec2_yr2)
    '''
    #df_mec_y2.to_csv('pdFile_MECcurmatch_last10m_norBzdom_Yr2.txt', sep = '\t', index = False)
    #df_imp_MECpos_yr2 = pd.read_csv('pdFile_MECcurmatch_last10m_norBzdom_Yr2.txt', sep = '\t')
    #compare_imp_df(df_mec_y2, df_imp_MECpos_yr2)

    #Exporting yr3 MEC posdata for each last10min interval and comparing the exported/OG dataframe
    #df_MECpos_yr3 = get_MEC_df(df_NSL10_yr3, yr3_MECposdata)
    #compare_mec_start_end(df_MECpos_yr3, df_NSL10_yr3)
    #For yr3_currdata(and mecdata): dropping NoData row
    #df_mec_y3 = df_MECpos_yr3.drop([113,114,115,272]) #yr3 norBz drop
    #df_mec_y3 = df_mec_y3.reset_index(drop = True) #reset index# column for new DF
    
    '''
    For norBz yr3_currdata
      #df_curr = df_currdata([113,114,115,272]) #yr3 norBz drop
    
    #For posBy yr3_currdata(and mecdata): dropping NoData row
    #df_mec_y3 = df_MECpos_yr3.drop([689, 690, 691, 692, 693, 694, 695, 696])# yr3 posBy drop
    
    #matching to soBz yr3 current df
    #df_mec2_yr3 = df_MECpos_yr3.drop([393, 394]) #dropping NoData row for Yr3
    #df_mec2_yr3 = df_mec2_yr3.reset_index(drop = True) #reset index# column for new DF
    #print('\n New Drop DF')
    #display(df_mec2_yr3)
    '''
    #df_mec_y3.to_csv('pdFile_MECcurmatch_last10m_norBzdom_Yr3.txt', sep = '\t', index = False)
    #df_imp_MECpos_yr3 = pd.read_csv('pdFile_MECcurmatch_last10m_norBzdom_Yr3.txt', sep = '\t')
    #compare_imp_df(df_mec_y3, df_imp_MECpos_yr3)

    #Exporting yr4 MEC posdata for each last10min interval and comparing the exported/OG dataframe
    #df_MECpos_yr4 = get_MEC_df(df_NSL10_yr4, yr4_MECposdata)
    #compare_mec_start_end(df_MECpos_yr4, df_NSL10_yr4)
    ''' 
    #For negBy yr4_currdata(and mecdata): dropping NoData row
    #df_mec_yr4 = df_MECpos_yr4.drop([896]) #yr4 negBy drop
    #df_mec_yr4 = df_mec_yr4.reset_index(drop = True) #reset index# column for new DF
    ''';
    #df_MECpos_yr4.to_csv('pdFile_MECcurmatch_last10m_norBzdom_Yr4.txt', sep = '\t', index = False)
    #df_imp_MECpos_yr4 = pd.read_csv('pdFile_MECcurmatch_last10m_norBzdom_Yr4.txt', sep = '\t')
    #compare_imp_df(df_MECpos_yr4, df_imp_MECpos_yr4)

    #Exporting yr5 MEC posdata for each last10min interval and comparing the exported/OG dataframe
    #df_MECpos_noMECgap_yr5 = get_MEC_df(df_NSL10_yr5, yr5_MECposdata)
    #compare_mec_start_end(df_MECpos_noMECgap_yr5, df_NSL10_yr5)
    #For yr5_currdata(and mecdata): dropping NoData Row
    #df_mec_yr5 = df_MECpos_noMECgap_yr5.drop([131]) # yr5 norBz drop
    #df_mec_yr5 = df_mec_yr5.reset_index(drop = True) #reset index# column for new DF

    '''
    For norBz yr5_currdata
    #df_curr = df_currdata.drop([131]) #yr5 norBz drop
    
      #For posBy yr5_currdata(and mecdata): dropping NoData Row
    #df_mec_yr5 = df_MECpos_noMECgap.drop([251, 252, 253]) # yr5 posBy drop
    
    #For negBy yr5_currdata(and mecdata): dropping NoData row
    df_mec_yr5 = df_MECpos_noMECgap_yr5.drop([232]) #yr5 negBy drop
    df_mec_yr5 = df_mec_yr5.reset_index(drop = True) #reset index# column for new DF
    ''';
    #df_mec_yr5.to_csv('pdFile_MECcurmatch_last10m_noGAP_norBzdom_Yr5.txt', sep = '\t', index = False)
    #df_imp_MECpos_yr5 = pd.read_csv('pdFile_MECcurmatch_last10m_noGAP_norBzdom_Yr5.txt', sep = '\t')
    #compare_imp_df(df_mec_yr5, df_imp_MECpos_yr5)
   

    return;

#Creating corresponding MECpos DFs for negBy CurrPeak Data
#checkin_MEC_dfs( df_nBy_noslash10m_yr1,  df_nBy_noslash10m_yr2,  df_nBy_noslash10m_yr3, 
#     df_nBy_noslash10m_yr4,  df_nBy_noslash10m_yr5)


#Creating corresponding MECpos DFs for posBy CurrPeak Data
#checkin_MEC_dfs( df_pBy_noslash10m_yr1,  df_pBy_noslash10m_yr2,  df_pBy_noslash10m_yr3, 
 #    df_pBy_noslash10m_yr4,  df_pBy_noslash10m_yr5)


#Creating corresponding MECpos DFs for norBz CurrPeak Data
#checkin_MEC_dfs( df_noBz_noslash10m_yr1,  df_noBz_noslash10m_yr2,  df_noBz_noslash10m_yr3, 
#     df_noBz_noslash10m_yr4,  df_noBz_noslash10m_yr5)


def checkin_MEC_dfs_NOalter(df_NSL10_yr1, df_NSL10_yr2, df_NSL10_yr3, df_NSL10_yr4, df_NSL10_yr5):
    """Checking newly made MEC position dataframes by ensuring date column arrays matches the
    General last10m Stable List. Then compare the exported/imported dataframes"""
    print('*General* Inputs which are used to create MEC DFs:\n', namestr(df_NSL10_yr1, globals()),
          namestr(df_NSL10_yr2, globals()),namestr(df_NSL10_yr3, globals()), '\n', 
          namestr(df_NSL10_yr4, globals()), namestr(df_NSL10_yr5, globals()))
    
    #Exporting yr1 MEC posdata for each last10min interval and comparing the exported/OG dataframe
    #df_MECpos_yr1 = get_MEC_df(df_NSL10_yr1, yr1_MECposdata)
    #compare_mec_start_end(df_MECpos_yr1, df_NSL10_yr1)
    #df_MECpos_yr1.to_csv('pdFile_MECmatch_last10m_general_Yr1.txt', sep = '\t', index = False)
    #df_imp_MECpos_yr1 = pd.read_csv('pdFile_MECmatch_last10m_general_Yr1.txt', sep = '\t')
    #compare_imp_df(df_MECpos_yr1, df_imp_MECpos_yr1)
    
     #Exporting yr2 MEC posdata for each last10min interval and comparing the exported/OG dataframe
    #df_MECpos_yr2 = get_MEC_df(df_NSL10_yr2, yr2_MECposdata)
    #compare_mec_start_end(df_MECpos_yr2, df_NSL10_yr2)
    #df_MECpos_yr2.to_csv('pdFile_MECmatch_last10m_general_Yr2.txt', sep = '\t', index = False)
    #df_imp_MECpos_yr2 = pd.read_csv('pdFile_MECmatch_last10m_general_Yr2.txt', sep = '\t')
    #compare_imp_df(df_MECpos_yr2, df_imp_MECpos_yr2)
    
     #Exporting yr3 MEC posdata for each last10min interval and comparing the exported/OG dataframe
    #df_MECpos_yr3 = get_MEC_df(df_NSL10_yr3, yr3_MECposdata)
    #compare_mec_start_end(df_MECpos_yr3, df_NSL10_yr3)
    #df_MECpos_yr3.to_csv('pdFile_MECmatch_last10m_general_Yr3.txt', sep = '\t', index = False)
    #df_imp_MECpos_yr3 = pd.read_csv('pdFile_MECmatch_last10m_general_Yr3.txt', sep = '\t')
    #compare_imp_df(df_MECpos_yr3, df_imp_MECpos_yr3)
    
     #Exporting yr4 MEC posdata for each last10min interval and comparing the exported/OG dataframe
    #df_MECpos_yr4 = get_MEC_df(df_NSL10_yr4, yr4_MECposdata)
    #compare_mec_start_end(df_MECpos_yr4, df_NSL10_yr4)
    #df_MECpos_yr4.to_csv('pdFile_MECmatch_last10m_general_Yr4.txt', sep = '\t', index = False)
    #df_imp_MECpos_yr4 = pd.read_csv('pdFile_MECmatch_last10m_general_Yr4.txt', sep = '\t')
    #compare_imp_df(df_MECpos_yr4, df_imp_MECpos_yr4)
    
    
     #Exporting yr5 MEC posdata for each last10min interval and comparing the exported/OG dataframe
    #df_MECpos_yr5 = get_MEC_df(df_NSL10_yr5, yr5_MECposdata)
    #compare_mec_start_end(df_MECpos_yr5, df_NSL10_yr5)
    #df_MECpos_yr5.to_csv('pdFile_MECmatch_last10m_noGAP_general_Yr5.txt', sep = '\t', index = False)
    #df_imp_MECpos_yr5 = pd.read_csv('pdFile_MECmatch_last10m_noGAP_general_Yr5.txt', sep = '\t')
    #compare_imp_df(df_MECpos_yr5, df_imp_MECpos_yr5)
  

    return;

#Creating corresponding MECpos DFs from general Yrly Stable Lists
#checkin_MEC_dfs_NOalter( df_gen_noslash10m_yr1,  df_gen_noslash10m_yr2,  df_gen_noslash10m_yr3, 
#     df_gen_noslash10m_yr4,  df_gen_noslash10m_yr5)

['rmag', 't', 'x', 'y', 'z']


* (11-23-2021) Comments on how to match tranges of MEC DF's onto their corresponding Current DF:
~~~
    matching to yr2 soBz current df
    #df_mec2_yr2 = df_MECpos_yr2.drop([189, 190, 191, 192, 193, 194, 195]) #dropping NoData row for Yr2
    #df_mec2_yr2 = df_mec2_yr2.reset_index(drop = True) #reset index# column for new DF
    #print('\n New drop DF')
    #display(df_mec2_yr2)

    matching to soBz yr3 current df
    #df_mec2_yr3 = df_MECpos_yr3.drop([393, 394]) #dropping NoData row for Yr3
    #df_mec2_yr3 = df_mec2_yr3.reset_index(drop = True) #reset index# column for new DF
    #print('\n New Drop DF')
    #display(df_mec2_yr3)
~~~
* From CurrentDF Export cell, *Comments about which tranges to drop for each Clockang Current Dataframe:*
~~~
'''
    For norBz yr5_currdata
    #df_curr = df_currdata.drop([131]) #yr5 norBz drop
      DropList index: [131.]
      ['2020-02-10/20:15:00', '2020-02-10/20:25:00']

    For norBz yr3_currdata
    #df_curr = df_currdata([113,114,115,272]) #yr3 norBz drop
        DropList index: [113. 114. 115. 272.]
        ['2018-05-27/04:50:00', '2018-05-27/05:00:00']
        ['2018-05-27/05:20:00', '2018-05-27/05:30:00']
        ['2018-05-27/21:40:00', '2018-05-27/21:50:00']
        ['2018-05-27/22:30:00', '2018-05-27/22:40:00']



    For norBz yr2_currdata
    #df_curr = df_currdata.drop([105]) #yr2 norBz drop
        DropList index: [105.]
        ['2017-01-04/16:40:00', '2017-01-04/16:50:00']


    For posBy yr5_currdata
    #df_curr = df_currdata.drop([251, 252, 253]) # yr5 posBy drop
        DropList index: [251. 252. 253.]

    For posBy yr3_currdata
    #df_curr = df_currdata.drop([689, 690, 691, 692, 693, 694, 695, 696])# yr3 posBy drop
        DropList index: [689. 690. 691. 692. 693. 694. 695. 696.]


    For posBy yr2_currdata
    #df_curr = df_currdata.drop([481, 482, 483, 484, 485, 486, 487, 488, 489, 490])# yr2 posBy drop


    For negBy yr5_currdata
    #df_curr = df_currdata.drop([232]) #yr5 negBy drop
        DropList index: [232.]
        ['2020-02-10/17:40:00', '2020-02-10/17:50:00']
    
    For negBy yr4_currdata
    #df_curr = df_currdata.drop([896]) #yr4 negBy drop
    
    For negBy yr2_currdata
    #df_curr = df_currdata.drop([234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244])# yr2 negBy drop
    #df_curr = df_curr.reset_index(drop=True)
    
    For SoBz yr2_currData
    #df_curr2_yr2 = df_currdata_yr2.drop([189, 190, 191, 192, 193, 194, 195]) #dropping NoData row for Yr2
    #df_curr2_yr2 = df_curr2_yr2.reset_index(drop = True) #reset index# column for new DF
    
    For SoBz yr3_currData
    #df_curr2_yr3 = df_currdata_yr3.drop([393, 394]) #dropping NoData row for Yr3
    #df_curr2_yr3 = df_curr2_yr3.reset_index(drop = True) #reset index# column for new DF
''';
~~~

### Section 1.2:
* Re-Importing trange syntax appropriate last10min Stable IMF dataframes and converting said dataframes into suitable trange tuples for looping

In [7]:
#Importing last10m soBz dataframes
'''
df_soBz_last10_yr1 = pd.read_csv(r'/Users/salinasha/Desktop/MacBoo Codes/Space Physics Codes/Under pySPEDAS_venv/'
         'Exported_trangeList_Last10min/Old_Exported_trange_Last10m/SoBz_trange_Last10m_v2/pdFile_Yr1_last10m_trange_soBzdom_v2.txt', sep = '\t')
df_soBz_last10_yr2 = pd.read_csv(r'/Users/salinasha/Desktop/MacBoo Codes/Space Physics Codes/Under pySPEDAS_venv/'
         'Exported_trangeList_Last10min/Old_Exported_trange_Last10m/SoBz_trange_Last10m_v2/pdFile_Yr2_last10m_trange_soBzdom_v2.txt', sep = '\t')
df_soBz_last10_yr3 = pd.read_csv(r'/Users/salinasha/Desktop/MacBoo Codes/Space Physics Codes/Under pySPEDAS_venv/'
         'Exported_trangeList_Last10min/Old_Exported_trange_Last10m/SoBz_trange_Last10m_v2/pdFile_Yr3_last10m_trange_soBzdom_v2.txt', sep = '\t') 
df_soBz_last10_yr4 = pd.read_csv(r'/Users/salinasha/Desktop/MacBoo Codes/Space Physics Codes/Under pySPEDAS_venv/'
         'Exported_trangeList_Last10min/Old_Exported_trange_Last10m/SoBz_trange_Last10m_v2/pdFile_Yr4_last10m_trange_soBzdom_v2.txt', sep = '\t')
df_soBz_last10_yr5 = pd.read_csv(r'/Users/salinasha/Desktop/MacBoo Codes/Space Physics Codes/Under pySPEDAS_venv/'
         'Exported_trangeList_Last10min/Old_Exported_trange_Last10m/SoBz_trange_Last10m_v2/pdFile_Yr5_last10m_trange_noMECgap_soBzdom_v2.txt', sep = '\t')

print(sorted(df_soBz_last10_yr5))
print('Re-imported soBz stable DFs')
#show1to5DF(df_soBz_last10_yr1, df_soBz_last10_yr2, df_soBz_last10_yr3, df_soBz_last10_yr4, df_soBz_last10_yr5)


#Importing last10m norBz dataframes
df_norBz_last10_yr1 = pd.read_csv(r'/Users/salinasha/Desktop/MacBoo Codes/Space Physics Codes/Under pySPEDAS_venv/'
         'Exported_trangeList_Last10min/Old_Exported_trange_Last10m/NorBz_trange_Last10m/pdFile_Yr1_last10m_trange_norBzdom.txt', sep = '\t')

df_norBz_last10_yr2 = pd.read_csv(r'/Users/salinasha/Desktop/MacBoo Codes/Space Physics Codes/Under pySPEDAS_venv/'
         'Exported_trangeList_Last10min/Old_Exported_trange_Last10m/NorBz_trange_Last10m/pdFile_Yr2_last10m_trange_norBzdom.txt', sep = '\t')
df_norBz_last10_yr3 = pd.read_csv(r'/Users/salinasha/Desktop/MacBoo Codes/Space Physics Codes/Under pySPEDAS_venv/'
         'Exported_trangeList_Last10min/Old_Exported_trange_Last10m/NorBz_trange_Last10m/pdFile_Yr3_last10m_trange_norBzdom.txt', sep = '\t')
df_norBz_last10_yr4 = pd.read_csv(r'/Users/salinasha/Desktop/MacBoo Codes/Space Physics Codes/Under pySPEDAS_venv/'
         'Exported_trangeList_Last10min/Old_Exported_trange_Last10m/NorBz_trange_Last10m/pdFile_Yr4_last10m_trange_norBzdom.txt', sep = '\t')
df_norBz_last10_yr5 = pd.read_csv(r'/Users/salinasha/Desktop/MacBoo Codes/Space Physics Codes/Under pySPEDAS_venv/'
         'Exported_trangeList_Last10min/Old_Exported_trange_Last10m/NorBz_trange_Last10m/pdFile_Yr5_last10m_trange_noMECgap_norBzdom.txt', sep = '\t')

print('Re-imported norBz Stable DFs')
#show1to5DF(df_norBz_last10_yr1, df_norBz_last10_yr2, df_norBz_last10_yr3, df_norBz_last10_yr4, df_norBz_last10_yr5)
''';


#Importing last10m posBy dataframes
df_posBy_last10_yr1 = pd.read_csv(r'/Users/salinasha/Desktop/MacBoo Codes/Space Physics Codes/Under pySPEDAS_venv/'
         'Exported_trangeList_Last10min/PosBy_withBx_trange_Last10m/pdFile_Yr1_last10m_trange_posByled_withBx.txt', sep = '\t')

df_posBy_last10_yr2 = pd.read_csv(r'/Users/salinasha/Desktop/MacBoo Codes/Space Physics Codes/Under pySPEDAS_venv/'
     'Exported_trangeList_Last10min/PosBy_withBx_trange_Last10m/pdFile_Yr2_last10m_trange_posByled_withBx.txt', sep = '\t')

df_posBy_last10_yr3 = pd.read_csv(r'/Users/salinasha/Desktop/MacBoo Codes/Space Physics Codes/Under pySPEDAS_venv/'
     'Exported_trangeList_Last10min/PosBy_withBx_trange_Last10m/pdFile_Yr3_last10m_trange_posByled_withBx.txt', sep = '\t')

df_posBy_last10_yr4 = pd.read_csv(r'/Users/salinasha/Desktop/MacBoo Codes/Space Physics Codes/Under pySPEDAS_venv/'
     'Exported_trangeList_Last10min/PosBy_withBx_trange_Last10m/pdFile_Yr4_last10m_trange_posByled_withBx.txt', sep = '\t')

df_posBy_last10_yr5 = pd.read_csv(r'/Users/salinasha/Desktop/MacBoo Codes/Space Physics Codes/Under pySPEDAS_venv/'
     'Exported_trangeList_Last10min/PosBy_withBx_trange_Last10m//pdFile_Yr5_last10m_trange_noMECgap_posByled_withBx.txt', sep = '\t')

print('Re-imported posBy Stable tranges')
#show1to5DF(df_posBy_last10_yr1, df_posBy_last10_yr2, df_posBy_last10_yr3, df_posBy_last10_yr4, df_posBy_last10_yr5)

#'''
#Importing last10m negBy dataframes
df_negBy_last10_yr1 = pd.read_csv(r'/Users/salinasha/Desktop/MacBoo Codes/Space Physics Codes/Under pySPEDAS_venv/'
     'Exported_trangeList_Last10min/NegBy_withBx_trange_Last10m/pdFile_Yr1_last10m_trange_negByled_withBx.txt', sep = '\t')

df_negBy_last10_yr2 = pd.read_csv(r'/Users/salinasha/Desktop/MacBoo Codes/Space Physics Codes/Under pySPEDAS_venv/'
     'Exported_trangeList_Last10min/NegBy_withBx_trange_Last10m/pdFile_Yr2_last10m_trange_negByled_withBx.txt', sep = '\t')

df_negBy_last10_yr3 = pd.read_csv(r'/Users/salinasha/Desktop/MacBoo Codes/Space Physics Codes/Under pySPEDAS_venv/'
     'Exported_trangeList_Last10min/NegBy_withBx_trange_Last10m/pdFile_Yr3_last10m_trange_negByled_withBx.txt', sep = '\t')

df_negBy_last10_yr4 = pd.read_csv(r'/Users/salinasha/Desktop/MacBoo Codes/Space Physics Codes/Under pySPEDAS_venv/'
     'Exported_trangeList_Last10min/NegBy_withBx_trange_Last10m/pdFile_Yr4_last10m_trange_negByled_withBx.txt', sep = '\t')

df_negBy_last10_yr5 = pd.read_csv(r'/Users/salinasha/Desktop/MacBoo Codes/Space Physics Codes/Under pySPEDAS_venv/'
     'Exported_trangeList_Last10min/NegBy_withBx_trange_Last10m/pdFile_Yr5_last10m_trange_noMECgap_negByled_withBx.txt', sep = '\t')

print('Re-imported negBy Stable tranges')
show1to5DF(df_negBy_last10_yr1, df_negBy_last10_yr2, df_negBy_last10_yr3, df_negBy_last10_yr4, df_negBy_last10_yr5)

#''';
#delete previously loaded tplot variables
pytplot.del_data() 


Re-imported posBy Stable tranges
Re-imported negBy Stable tranges


,Start_trang,End_trang,Clang_avg,Bx_avg(nT)
0,2015-09-01/00:25:00,2015-09-01/00:35:00,270.271,3.502
1,2015-09-01/08:55:00,2015-09-01/09:05:00,284.483,1.962
2,2015-09-01/10:50:00,2015-09-01/11:00:00,260.121,3.143
3,2015-09-01/12:45:00,2015-09-01/12:55:00,299.448,3.513
4,2015-09-01/19:30:00,2015-09-01/19:40:00,251.441,3.822
...,...,...,...,...
660,2016-05-30/09:20:00,2016-05-30/09:30:00,271.550,2.197
661,2016-05-30/09:50:00,2016-05-30/10:00:00,275.711,3.940
662,2016-05-30/10:20:00,2016-05-30/10:30:00,267.143,3.412
663,2016-05-30/13:20:00,2016-05-30/13:30:00,266.897,3.228


,Start_trang,End_trang,Clang_avg,Bx_avg(nT)
0,2016-09-08/16:15:00,2016-09-08/16:25:00,254.787,-1.043
1,2016-09-11/22:30:00,2016-09-11/22:40:00,240.055,-2.332
2,2016-09-11/23:00:00,2016-09-11/23:10:00,237.653,-4.373
3,2016-09-16/13:55:00,2016-09-16/14:05:00,305.148,2.902
4,2016-09-17/17:40:00,2016-09-17/17:50:00,302.025,2.752
...,...,...,...,...
608,2017-05-27/10:45:00,2017-05-27/10:55:00,274.884,-0.223
609,2017-05-27/11:15:00,2017-05-27/11:25:00,278.537,-0.262
610,2017-05-27/11:45:00,2017-05-27/11:55:00,268.346,0.222
611,2017-05-27/14:30:00,2017-05-27/14:40:00,264.697,-1.552


,Start_trang,End_trang,Clang_avg,Bx_avg(nT)
0,2017-09-01/18:50:00,2017-09-01/19:00:00,311.273,-4.432
1,2017-09-01/20:45:00,2017-09-01/20:55:00,279.942,-5.642
2,2017-09-01/22:40:00,2017-09-01/22:50:00,310.823,-5.735
3,2017-09-03/05:45:00,2017-09-03/05:55:00,309.260,-3.452
4,2017-09-06/23:00:00,2017-09-06/23:10:00,295.288,-0.320
...,...,...,...,...
817,2018-05-22/14:30:00,2018-05-22/14:40:00,265.307,4.483
818,2018-05-22/15:00:00,2018-05-22/15:10:00,265.247,4.557
819,2018-05-22/16:30:00,2018-05-22/16:40:00,271.157,4.250
820,2018-05-24/17:50:00,2018-05-24/18:00:00,263.469,-3.033


,Start_trang,End_trang,Clang_avg,Bx_avg(nT)
0,2018-09-01/01:35:00,2018-09-01/01:45:00,284.177,-3.255
1,2018-09-01/04:00:00,2018-09-01/04:10:00,312.375,-2.530
2,2018-09-02/09:15:00,2018-09-02/09:25:00,308.625,1.515
3,2018-09-02/09:45:00,2018-09-02/09:55:00,313.785,1.790
4,2018-09-02/11:45:00,2018-09-02/11:55:00,302.135,1.747
...,...,...,...,...
892,2019-05-24/12:55:00,2019-05-24/13:05:00,278.693,1.618
893,2019-05-24/13:25:00,2019-05-24/13:35:00,265.719,1.315
894,2019-05-28/16:10:00,2019-05-28/16:20:00,299.266,0.162
895,2019-05-28/20:45:00,2019-05-28/20:55:00,236.574,5.833


,Start_trang,End_trang,Clang_avg,Bx_avg(nT)
0,2019-09-17/07:30:00,2019-09-17/07:40:00,284.799,-0.280
1,2019-09-17/15:15:00,2019-09-17/15:25:00,294.556,2.822
2,2019-09-17/15:45:00,2019-09-17/15:55:00,294.403,2.950
3,2019-09-17/16:40:00,2019-09-17/16:50:00,235.839,3.665
4,2019-09-20/09:50:00,2019-09-20/10:00:00,251.279,-1.085
...,...,...,...,...
518,2020-05-30/10:50:00,2020-05-30/11:00:00,267.097,5.738
519,2020-05-30/11:20:00,2020-05-30/11:30:00,270.261,5.477
520,2020-05-30/12:15:00,2020-05-30/12:25:00,278.151,4.732
521,2020-05-30/13:35:00,2020-05-30/13:45:00,288.296,4.653


In [10]:
def convert_trange_tuple(df_yrX):
#Converting last10 dataframes into suitable trange tuple lists
    startSTR = df_yrX['Start_trang'].to_numpy(copy=True)
    endSTR = df_yrX['End_trang'].to_numpy(copy=True)
    test = tuple() #creating empty tuple
    #appending list-items to empty tuple with for loop
    for i in range(len(startSTR)):
        tdate = [startSTR[i], endSTR[i]] #time intervals in proper trange syntax
        test = test + (tdate,)
    print(type(test)) #print(test[0])
    print(test[0], '--->', test[-1])
    print(len(test))
    return(test);
'''
print('SoBz trange tuple:')
trange_soBztuple_yr1 = convert_trange_tuple(df_soBz_last10_yr1)
trange_soBztuple_yr2 = convert_trange_tuple(df_soBz_last10_yr2)
trange_soBztuple_yr3 = convert_trange_tuple(df_soBz_last10_yr3)
trange_soBztuple_yr4 = convert_trange_tuple(df_soBz_last10_yr4)
trange_soBztuple_yr5 = convert_trange_tuple(df_soBz_last10_yr5)

print('----\nNorBz trange tuple:')
trange_norBztuple_yr1 = convert_trange_tuple(df_norBz_last10_yr1)
trange_norBztuple_yr2 = convert_trange_tuple(df_norBz_last10_yr2)
trange_norBztuple_yr3 = convert_trange_tuple(df_norBz_last10_yr3)
trange_norBztuple_yr4 = convert_trange_tuple(df_norBz_last10_yr4)
trange_norBztuple_yr5 = convert_trange_tuple(df_norBz_last10_yr5)
'''
print('----\nNegBy trange tuple:')
trange_negBytuple_yr1 = convert_trange_tuple(df_negBy_last10_yr1)
trange_negBytuple_yr2 = convert_trange_tuple(df_negBy_last10_yr2)
trange_negBytuple_yr3 = convert_trange_tuple(df_negBy_last10_yr3)
trange_negBytuple_yr4 = convert_trange_tuple(df_negBy_last10_yr4)
trange_negBytuple_yr5 = convert_trange_tuple(df_negBy_last10_yr5)

print('----\nPosBy trange tuple:')
trange_posBytuple_yr1 = convert_trange_tuple(df_posBy_last10_yr1)
trange_posBytuple_yr2 = convert_trange_tuple(df_posBy_last10_yr2)
trange_posBytuple_yr3 = convert_trange_tuple(df_posBy_last10_yr3)
trange_posBytuple_yr4 = convert_trange_tuple(df_posBy_last10_yr4)
trange_posBytuple_yr5 = convert_trange_tuple(df_posBy_last10_yr5)
#''';

----
NegBy trange tuple:
<class 'tuple'>
['2015-09-01/00:25:00', '2015-09-01/00:35:00'] ---> ['2016-05-30/14:30:00', '2016-05-30/14:40:00']
665
<class 'tuple'>
['2016-09-08/16:15:00', '2016-09-08/16:25:00'] ---> ['2017-05-30/07:40:00', '2017-05-30/07:50:00']
613
<class 'tuple'>
['2017-09-01/18:50:00', '2017-09-01/19:00:00'] ---> ['2018-05-29/19:30:00', '2018-05-29/19:40:00']
822
<class 'tuple'>
['2018-09-01/01:35:00', '2018-09-01/01:45:00'] ---> ['2019-05-29/05:20:00', '2019-05-29/05:30:00']
897
<class 'tuple'>
['2019-09-17/07:30:00', '2019-09-17/07:40:00'] ---> ['2020-05-30/14:05:00', '2020-05-30/14:15:00']
523
----
PosBy trange tuple:
<class 'tuple'>
['2015-09-03/00:00:00', '2015-09-03/00:10:00'] ---> ['2016-05-30/12:10:00', '2016-05-30/12:20:00']
831
<class 'tuple'>
['2016-09-01/00:15:00', '2016-09-01/00:25:00'] ---> ['2017-05-27/18:00:00', '2017-05-27/18:10:00']
813
<class 'tuple'>
['2017-09-01/11:15:00', '2017-09-01/11:25:00'] ---> ['2018-05-30/18:20:00', '2018-05-30/18:30:00']
72

* Effective way of appending items no an empty tuple where the desired elements are lists
    * https://stackoverflow.com/questions/16730339/python-add-item-to-the-tuple

# Section 2
## Extracting Current Data by looping through FGM and CURL data throughout several 10min intervals
* takes on avg ~32.7sec to load 10 min of FGM data and compute CURL params
* To loop with pySPEDAS functions, the trange intervals must be placed in a tuple
---
## (10-28-2021)
### Listing Currents from (ClockAng) Sorted Events Redux
#### Taking mean, median, and std of current peaks instead of full data to filter our 'noisy' current
* **Looks for peaks from the current magnitude then extrapolate those locations to the mean,med, and std values of the curlB components and divB**
* also, name file that mentions what the peak prominence, or relative height, was. Which in this case was 0.1(or relative height >=0.1) or 0.08 orwhatever I decide on
    * Potential idea: look for max curr_mag value in a dataset interval. If max > 0.1, set prominence to 0.09(record peaks whose rel height >=0.1). If max < 0.1, set prominence to 0.07(record peaks whose rel. height >= 0.08
#### Excerpt from Birdwatchin 2 Script:
 * (10-14-2021) As discovered here[Birdwatchin 2] and discussed in great deal with Rick. The original current data I extracted from the SoBz STable intervals took the mean, median, and std of the interval's **entire** data set. Which caused those statistical parameters to center around the 'noisy' part of the data. To remedy this, I will modify the `Mapping Currents Pt1` to capture the time interval stat's of the **current peaks** instead of the entire data set using `scipy_find_peaks`
     * height parameter: I search for peaks above the avg_value since the original algorithm's stat parameters captured the noise's location quite nicely. As verified in both the above plot[Birdwatchin 2] and currstat plots I made in `Mapping Currents pt2`
     * prominence parameter: set minimum relative peak height to 0.1 since most current peaks(observations from old BirdWatchin script) in steady magsheath were usually around 0.3uA/m2
***     


### Section 2.1:
#### Loading/Looping through  FGM and Curlometer Data(GSM Coord)
* discovered FGM survey data points are made every 0.12 to 0.13seconds    
* (9-18-2021) In plotting out individual time interval's curlB/mu0 data, I discovered that the current components contain nan-values which messes up average and median calculations. Luckily, `np.nanmedian` and `np.nanmean` exist to deter that issue
    * https://stackoverflow.com/questions/36224066/average-of-a-numpy-array-returns-nan
    
* (1-13-2022)
    * As discussed with Tony at AGU 2021, the standard deviation is a good way of capturing the location of noise
    * For current peak extraction, I'm ensuring that peaks are recorded above 2_std
        * I noticed in `Bird Watchin 2`, this 2_std was sometimes comparable too or higher than the avg_value. Which helps ensure that I'm capturing significant peaks well above the noisy part of the data

In [9]:
trange_test = ['2015-09-01/10:05:00', '2015-09-01/10:15:00']
trange_test2 = ['2015-09-01/10:00:00', '2015-09-01/10:15:00']

print(trange_test)
print(trange_test[0])
print(type(trange_test[0]))
print(trange_soBztuple_yr1[0])

['2015-09-01/10:05:00', '2015-09-01/10:15:00']
2015-09-01/10:05:00
<class 'str'>


NameError: name 'trange_soBztuple_yr1' is not defined

In [ ]:
###Loading up mec position, fgm B-field data, and Computing Curlometer Parameters#####################
##Loading and plotting 'srvy' data is fine for all
def getFGM_data(achDate):
# Loading up fgm B-field and mec position data with string inputted date list in GSM coord
    pytplot.del_data() #deleting previously loaded tplot variables
    print('\nInput date was: %s '%(achDate))
    fgm_vars = fgm(probe=[1, 2, 3, 4], trange=achDate, time_clip=True, varformat='*_gsm_*')
        #default data rate is 'srvy' for fgm routine    
    mec_vars = mec(probe=[1, 2, 3, 4], trange = achDate, time_clip=True) 
    #e_tempar_time, e_tempar_vals = get_data('mms1_des_temppara_fast')
    bgsm_time, bgsm_vals = get_data('mms1_fgm_b_gsm_srvy_l2')
    #Defining string variables for FGM start/end time
    achFGM_start = time_string(bgsm_time[0])
    achFGM_end = time_string(bgsm_time[-1])
    print('Confirming loaded FGM data range: %s to %s' %(achFGM_start[0:20], achFGM_end[0:20]))
    return(bgsm_time);
    # takes about ~27sec to load 10min of data

#start = time.perf_counter()
##getFGM_data(trange_test2)
#end = time.perf_counter()
#print('Time to load FGM an MEC data" %0.2f s'%(end-start))


def curl_params():
# Computing CURL parameters with loaded fgm data in GSM coord
    #defining curlometer inputs
    mec_pos = ['mms1_mec_r_gsm', 'mms2_mec_r_gsm', 'mms3_mec_r_gsm', 'mms4_mec_r_gsm']
    prob_fields = ['mms1_fgm_b_gsm_srvy_l2_bvec', 'mms2_fgm_b_gsm_srvy_l2_bvec', 
          'mms3_fgm_b_gsm_srvy_l2_bvec', 'mms4_fgm_b_gsm_srvy_l2_bvec']
    #Computing curlometer parameters
    curlometer_vars = curlometer(fields=prob_fields, positions=mec_pos)
    #print('Time to load 10min of curlometer data: %0.2f s'%(end_curl-start_curl))
    #takes ~20min for one day, ~4.49s for 10min, and ~6.64s for 15min of data
        #curlometer params: tplot(['divB', 'curlB', 'jtotal', 'jperp', 'jpar', 'baryb'])
        ##units of curlB and divB are in nT/km
    #tplot_names()
    return;

def plot_fgm_curl():
## Plotting fgm B-field and curlom parameters
    #tplot(['mms1_fgm_b_gse_srvy_l2','mms1_fgm_b_gse_srvy_l2_bvec', 'mms1_fgm_b_gse_srvy_l2_btot'])
    tplot(['mms1_fgm_b_gsm_srvy_l2_bvec','curlB', 'divB', 'jtotal'])
    #tplot_names();
    return;

def get_curlndivB(mms1_fgm_bgsm_time):
# Creating/extracting new tplot variables for curlB and divB in terms of current units#####
    #defining constants from the NRL Plasma Foundry
    R_e = 6371 #km, Earth radius
    mu0 = 4e-7*np.pi #H/m == Tm/A, permeability of free space
        #mu0*J == [Tm/A *A/m2] == [T/m] == curlB
            #1km = 1000m and 1nT = 1e-9T
        #mu0[Tm/A]*(1km/1000m)*(1nT/1e-9T) == mu0(nT*km/A)*1e+6
    mu0_nT_km = mu0*1e+6 #nT*km/A
    
    #Extracting curlB/divB(unit = nT/km) data to create |curlB|/mu0 and divB/mu0(unit = A/km2 or uA/m2) tplot vars
    curlb_time, curlb_vec = get_data('curlB')
    divb_time, divb_vals = get_data('divB')
    print('\n', type(curlb_vec))
    print('curlB shape:', np.shape(curlb_vec)) # or curlb_vec.shape)
    curlb_mag = np.sqrt(curlb_vec[:,0]**2 + curlb_vec[:,1]**2 + curlb_vec[:,2]**2)
        #print('Does curlb and divb time arrays match', np.array_equal(curlb_time, divb_time)) #they match
        #store_data('curlB_mag_mu0', data={'x': curlb_time, 'y': curlb_mag/mu0_nT_km})
        #store_data('divB_mu0', data={'x': divb_time, 'y': divb_vals/mu0_nT_km})
        #Creating single tplot variable for |curlB|/mu0 and divB/mu0
        #join_vec(['curlB_mag_mu0', 'divB_mu0'], new_tvar='curlB_divB_mu0')
        #Displaying curlB start-end dates and confirming if it matches with prob1 fgm time array
        #print('Curlb time:\n', time_string(curlb_time[:5]), '\n', time_string(curlb_time[-5:]))
    print('FGM Bgsm Prob1 time and curlB time arrays match:', np.array_equal(curlb_time, mms1_fgm_bgsm_time))
    # Plotting curlb and divb components to display statistical features
    #plot_currents(curlb_time, curlb_vec[:,0], curlb_vec[:,1], curlb_vec[:,2], curlb_mag, divb_vals)
    # Return single arrays vector of current stats for each component
    current_stats = get_curr_stats(curlb_time, curlb_vec[:,0], curlb_vec[:,1], curlb_vec[:,2], curlb_mag, divb_vals)
    return(current_stats);
    #return (curlb_time, curlb_vec[:,0], curlb_vec[:,1], curlb_vec[:,2], curlb_mag, divb_vals);


def plot_currents(curlb_utime, curlbx, curlby, curlbz, curlb_mag, divb):
# Plots curlB/mu0 in proper units(uA/m2) and displays parameters like median, mean, and what not
    mu0 = 4e-7*np.pi #H/m == Tm/A, permeability of free space
    mu0_nT_km = mu0*1e+6 #nT*km/A
    #Defining string variables for CURL start and end time
    achStart = time_string(curlb_utime[0])
    achEnd = time_string(curlb_utime[-1])
    achTRange = 'from %s to %s' %(achStart[0:19], achEnd[0:19])
    #Extract array of indices which yields Current_mag peak locations; whose relative height is >= 0.1
    peakmag = get_currpeaks(curlb_mag/mu0_nT_km)

    #Plotting curlb and divb components
    BxTitle = 'curlB_x/mu0(uA/m2) vs Unix Time for %s' %(achTRange)
    PlotCurr2(curlb_utime, curlbx/mu0_nT_km, peakmag, BxTitle)
    
    ByTitle = 'curlB_y/mu0(uA/m2) vs Unix Time for %s' %(achTRange)
    PlotCurr2(curlb_utime, curlby/mu0_nT_km, peakmag, ByTitle)
    
    BzTitle = 'curlB_z/mu0(uA/m2) vs Unix Time for %s' %(achTRange)
    PlotCurr2(curlb_utime, curlbz/mu0_nT_km, peakmag, BzTitle)
    
    BmagTitle = 'curlB_mag/mu0(uA/m2) vs Unix Time for %s' %(achTRange)
    PlotCurr2(curlb_utime, curlb_mag/mu0_nT_km, peakmag, BmagTitle)
    
    BdivTitle = 'divB/mu0(uA/m2) vs Unix Time for %s' %(achTRange)
    PlotCurr2(curlb_utime, divb/mu0_nT_km, peakmag, BdivTitle)
    return;

def get_curr_stats(curlb_utime, curlbx, curlby, curlbz, curlb_mag, divb):
# Compute/return mean, median, and std for each parameter input in current units(uA/m2)
    mu0 = 4e-7*np.pi #H/m == Tm/A, permeability of free space
    mu0_nT_km = mu0*1e+6 #nT*km/A
    #Defining string variables for CURL start and end time
    achStart = time_string(curlb_utime[0])
    achEnd = time_string(curlb_utime[-1])
    #Extract array of indices which yields Current_mag peak(0.1 prominence) locations
    peakmag = get_currpeaks(curlb_mag/mu0_nT_km)
    #Current arrays contain nan-values
    [Jx_avg, Jx_med, Jx_std] = get_stats(curlbx[peakmag]/mu0_nT_km)
    [Jy_avg, Jy_med, Jy_std] = get_stats(curlby[peakmag]/mu0_nT_km)
    [Jz_avg, Jz_med, Jz_std] = get_stats(curlbz[peakmag]/mu0_nT_km)
    [Jmag_avg, Jmag_med, Jmag_std] = get_stats(curlb_mag[peakmag]/mu0_nT_km)
    [divB_avg, divB_med, divB_std] = get_stats(divb[peakmag]/mu0_nT_km)
    #defining row vector for current stat parameters
    curr_stat_vec = [achStart[0:19], Jx_avg, Jy_avg, Jz_avg, Jmag_avg, divB_avg, Jx_med, Jy_med, Jz_med,
                Jmag_med, divB_med, Jx_std, Jy_std, Jz_std, Jmag_std, divB_std, achEnd[0:19]]
    return (curr_stat_vec);
    
    #['Cur_start','Jx_avg(uA/m2)', 'Jy_avg','Jz_avg','Jmag_avg','divB_avg',
    #'Jx_med(uA/m2)', 'Jy_med', 'Jz_med', 'Jmag_med', 'divB_med', 'Jx_std(uA/m2)',
     #'Jy_std', 'Jz_std', 'Jmag_std', 'divB_std','Cur_end']

def get_stats(afData):
# Return mean, median, and std of inputted data array
    #Turns out current arrays contain nan-values
    avg_value = np.nanmean(afData) #Computes average of inputted data array
    med_value = np.nanmedian(afData) #Computes median of inputteed data array
    std_value = np.nanstd(afData)
    return(avg_value, med_value, std_value);

def get_currpeaks(afCurrMag):
    """ Extract current peaks location from (curlB/muo)_mag then extrapolate those peak locations to the
    other current components after returning them"""
    # Recall that current arrays contain nan-values
    #avg_value = np.nanmean(afCurrMag) #Computes average of inputted data array
    std_value = np.nanstd(afCurrMag) #Computes std of inputted data array
     # find current peaks(excluding nan) above 2*std (old: avg_value) with a relative height >= 0.8
    peaks, _ = find_peaks(afCurrMag[~np.isnan(afCurrMag)], height = 2*std_value, prominence = 0.07)
    return(peaks);

def PlotCurr2(aftime, afData, peakmag, title):
    """Plots particular current component vs time and displays statistical features alongside location
    of peaks that correspond to current_mag peaks"""
    # Turns out current arrays contain nan-values
    avg_value = np.nanmean(afData) #Computes average of inputted data array
    med_value = np.nanmedian(afData) #Computes median of inputteed data array
    std_value = np.nanstd(afData)
    # Display and plot current: mean, median, peak, and raw data
    plt.figure(figsize = (7,7))
    plt.plot(aftime/1e+9, afData, 'g.') #raw data
    plt.plot(aftime[peakmag]/1e+9, afData[peakmag], 'ro', ms=6, label = 'Peak') #peak data
    print('Med and avg peak amplitudes are respectively %0.4f and  %0.4f'%(np.nanmedian(afData[peakmag]),
                                                                    np.nanmean(afData[peakmag])))
    print('Med value is %0.5f and avg value is %0.5f'%(med_value, avg_value))
    plt.axhline(avg_value,color = 'orange', linestyle = '--')
    plt.axhline(med_value,color = 'purple', linestyle = '--')
    plt.title('%s' %title, fontsize = 12)
    plt.legend(labels=['Std = %0.6f' %std_value, 'Peak-Mag','Average = %0.6f' %avg_value, 'Median = %0.6f' %med_value],
               loc = 'best', prop={'size': 10});
    return;

In [ ]:
#################Function for Looping through FGM and CURL params#################
#define test tuple whose elements are trange lists
achInput_test = (['2015-09-01/10:05:00', '2015-09-01/10:15:00'],['2015-09-01/11:40:00','2015-09-01/11:50:00'],
    ['2015-09-05/12:45:00','2015-09-05/12:55:00'], ['2015-09-07/18:00:00','2015-09-07/18:10:00'])
    
def current_single_date(trange):
#Measures the time it takes to load FGM 
#and compute CURL parameters for the corresponding trange input
    pytplot.del_data()
    start = time.perf_counter()
    mms1_fgm_bgsm_time = getFGM_data(trange)
    curl_params()
    curr_stat_vec = get_curlndivB(mms1_fgm_bgsm_time)
        #curr_stat_vec = [achStart[0:19], Jx_avg, Jy_avg, Jz_avg, Jmag_avg, divB_avg, Jx_med, Jy_med, Jz_med,
                #Jmag_med, divB_med, Jx_std, Jy_std, Jz_std, Jmag_std, divB_std, achEnd[0:19]]
    #plot_fgm_curl()
    end = time.perf_counter()
    print('Time it took to load 10min of FGM data and compute CURL params: %0.2f s'%(end-start))
    return((end-start), curr_stat_vec);

#timing_function()
#print(achInput_test[0])
#print(len(achInput))

def current_data_tuples(Date, error_vec):
#Measures the avg and total time it takes to load/compute FGM data with CURL params using inputted
#tuple of trange lists
    #creating empty dataframe with column names and defined index length as # of date intervals
    df_currdata = pd.DataFrame(columns = ['Cur_start','Jx_avg(uA/m2)', 'Jy_avg','Jz_avg','Jmag_avg','divB_avg',
                                    'Jx_med(uA/m2)', 'Jy_med', 'Jz_med', 'Jmag_med', 'divB_med', 'Jx_std(uA/m2)',
                                    'Jy_std', 'Jz_std', 'Jmag_std', 'divB_std','Cur_end'], index = np.arange(len(Date)))
    # initialized empty array and variable
    droplist = []
    fSum = 0
    for i in range(len(Date)):
        try: #method of bypassing errors that may occur from CURL computations(no data available)
            [com_time, curr_stat_vec] = current_single_date(Date[i]) #extracting runtime & current data from single trange
            fSum = fSum + com_time
        except: #add error_vec to DF row if error is detected
            print('DataError occurred with CURL or some other routine')
            df_currdata.iloc[i] = error_vec  # add error vector to DF row-index
            droplist = np.append(droplist,i) #add error element-index to array
        else:
            df_currdata.iloc[i] = curr_stat_vec  #adding current data to DF row-index
        #print(curr_stat_vec)
    
    display(df_currdata) #display current dataframe
    #displaying tranges in which data errors occurred over 
    if len(droplist) > 0:
        print('DropList index:', droplist)
        for i in range(len(droplist)):
            print(Date[int(droplist[i])])  # displays Drop tuple element index with index recorded by drop list
                                           # int(droplist[i]) ensures input into Date is an integer to avoid error
    else:
        print('No data errors detected for any tranges this year')
    print('\nTotal time to load/compute %d 10min intervals of FGM data and CURL params: %0.2f'%(len(Date), fSum))
    print('Avg time to load %d 10min intervals of FGM data and compute CURL params: %0.2f s'%(len(Date),fSum/len(Date)))
    return(df_currdata, droplist);

############################ Main Calling Functions Below #######################


#Defining array of strings with same length as DF Current Data row in case of detected errors
anList = np.array([])
anTEST = np.zeros(17)
print(len(anTEST), anTEST)
for i in range(len(anTEST)):
    anList = np.append(anList, 'NoData')
print(anList)
    

#time_test(trange_tuple_yr1)
print('Extracting CurrentData from negBy Stable tranges')
#[love, hope] = current_single_date(trange_soBztuple_yr5[0])
#[test_df, testdrop] = current_data_tuples(trange_soBztuple_yr5[:3], anList)

## Extracting Current Data from Yr1 Last10min List
#[df_negBycurrdata_yr1, drop_yr1] = current_data_tuples(trange_negBytuple_yr1, anList)

## Extracting Current Data from Yr2 Last10min List
#[df_negBycurrdata_yr2, drop_yr2] = current_data_tuples(trange_negBytuple_yr2, anList)

## Extracting Current Data from Yr3 Last10min List
#[df_negBycurrdata_yr3, drop_yr3] = current_data_tuples(trange_negBytuple_yr3, anList)

## Extracting Current Data from Yr4 Last10min List
#[df_negBycurrdata_yr4, drop_yr4] = current_data_tuples(trange_negBytuple_yr4, anList)

## Extracting Current Data from Yr5 Last10min List
#[df_negBycurrdata_yr5, drop_yr5] = current_data_tuples(trange_negBytuple_yr5, anList)



#curlbx, curlby, curlbz, curlb_mag, divb

### Section 2.2:
#### Testing and Exporting Newly Created Current Dataframes
1. Checking if time arrays in currDF match onto inputted time intervals
2. Removing rows whose elements contain 'NoData' due to the errors that were encountered in the extraction algorithm

In [ ]:
def compare_currstart(df_curr, df_last10_noslash):
#Compares dataframes columns for Start-dates between OG NO-slashlast10min and CurrData DFs
#They should be the same
    #Extracting and creating copies of DF start-date column arrays
    currSTR_start = df_curr['Cur_start'].to_numpy(copy=True)
    last10_start = df_last10_noslash['Start_str'].to_numpy(copy=True)
    print('CurrData and Last10m Dataframes start-date columns are the same: ',(currSTR_start == last10_start).all()) 
          #np.array_equal(currSTR_start, last10_start))
          #(currSTR_start == last10_start).all())
    #print((currSTR_start == last10_start).all())
    #print('Using np.array_equals, the start-date columns are theh same: ', np.array_equal(currSTR_start, last10_start))
    display(df_curr.iloc[:3])
    display(df_last10_noslash[:3])
    #COmparing individual elements to check for specific non-matching elements
    n = 0
    for i in range(len(currSTR_start)):
        if currSTR_start[i] == last10_start[i]:
            n = n+1
        else:
            print("Element %d Didn't match: "%i,currSTR_start[i], 'and', last10_start[i])
    print('Witih a length of %d, %d start-date elements match between Currdata and Last10'%(len(last10_start),n))
    return;

def checking_curr_dfs(df_currdata, df_noslash10, anDrop, trange_tuple):
    """Analyzing the newly made current datafrane by ensuring that it's start-date column array matches
    the trange input and lastly comparing the exported/imported dataframes"""
    print(sorted(df_noslash10)) #display noSlash10 column names
    print('DFs being analyzed:\n', namestr(df_currdata, globals()),namestr(df_noslash10, globals()),
         namestr(anDrop, globals()), namestr(trange_tuple, globals()))
    #Displaying errors that occurred throughout any tranges
    if len(anDrop) > 0:
        print('DropList index:', anDrop)
        for i in range(len(anDrop)):
            print(trange_tuple[int(anDrop[i])])  
            # displays Drop tuple element index with index recorded by drop list
            # int(droplist[i]) ensures input into Date is an integer to avoid error
    else:
        print('No data errors detected for any tranges this year\n')
    #display(df_currdata)
        #Checking yrX currData start-date columns
    compare_currstart(df_currdata, df_noslash10)
    #df_curr = df_currdata.drop([131]) #yr5 norBz drop

    #df_curr = df_curr.reset_index(drop=True)
    #display(df_curr)

    #compare_start(df_curr, df_noslash10)
        #Exporting yrX Current data for each last10min interval and comparing the exported/OG dataframe
    #df_currdata.to_csv('pdFile_CurrPeak_norBzdom_prom07_Yr4.txt', sep = '\t', index = False)
    #df_imp_currdata = pd.read_csv('pdFile_CurrPeak_norBzdom_prom07_Yr4.txt', sep = '\t')
    #compare_imp_df(df_currdata, df_imp_currdata)
    '''
    Exported currData DF file name template:
        pdFile_CurrPeak_negByled_prom07_Yr3.txt
        pdFile_CurrPeak_noMECgap_negByled_prom07_Yr5.txt
    ''';  
    #Exporting yr5 Current data for each last10min interval and comparing the exported/OG dataframe
    #compare_start(df_currdata, df_noslash10)
    #df_curr.to_csv('pdFile_CurrPeak_noMECgap_norBzdom_prom07_Yr5.txt', sep = '\t', index = False)
    #df_imp_currdata = pd.read_csv('pdFile_CurrPeak_noMECgap_norBzdom_prom07_Yr5.txt', sep = '\t')
    #compare_imp_df(df_curr, df_imp_currdata)
    
    '''
      For norBz yr5_currdata
      #df_curr = df_currdata.drop([131]) #yr5 norBz drop
          DropList index: [131.]
          ['2020-02-10/20:15:00', '2020-02-10/20:25:00']
    
    
    For norBz yr3_currdata
    #df_curr = df_currdata.drop([113,114,115,272]) #yr3 norBz drop
          DropList index: [113. 114. 115. 272.]
          ['2018-05-27/04:50:00', '2018-05-27/05:00:00']
          ['2018-05-27/05:20:00', '2018-05-27/05:30:00']
          ['2018-05-27/21:40:00', '2018-05-27/21:50:00']
          ['2018-05-27/22:30:00', '2018-05-27/22:40:00']
    
    
    For norBz yr2_currdata
    #df_curr = df_currdata.drop([105]) #yr2 norBz drop
        DropList index: [105.]
        ['2017-01-04/16:40:00', '2017-01-04/16:50:00']
    
    
    For posBy yr5_currdata
    #df_curr = df_currdata.drop([251, 252, 253]) # yr5 posBy drop
        DropList index: [251. 252. 253.]
        ['2020-02-10/21:10:00', '2020-02-10/21:20:00']
        ['2020-02-10/22:55:00', '2020-02-10/23:05:00']
        ['2020-02-10/23:25:00', '2020-02-10/23:35:00']
    
    
    For posBy yr3_currdata
    #df_curr = df_currdata.drop([689, 690, 691, 692, 693, 694, 695, 696])# yr3 posBy drop
        DropList index: [689. 690. 691. 692. 693. 694. 695. 696.]
        ['2018-05-27/14:20:00', '2018-05-27/14:30:00']
        ['2018-05-27/14:50:00', '2018-05-27/15:00:00']
        ['2018-05-27/16:55:00', '2018-05-27/17:05:00']
        ['2018-05-27/17:30:00', '2018-05-27/17:40:00']
        ['2018-05-27/18:00:00', '2018-05-27/18:10:00']
        ['2018-05-27/18:30:00', '2018-05-27/18:40:00']
        ['2018-05-27/19:25:00', '2018-05-27/19:35:00']
        ['2018-05-27/20:10:00', '2018-05-27/20:20:00']

    For posBy yr2_currdata
    #df_curr = df_currdata.drop([481, 482, 483, 484, 485, 486, 487, 488, 489, 490])# yr2 posBy drop
    
    For negBy yr5_currdata
    #df_curr = df_currdata.drop([232]) #yr5 negBy drop
        DropList index: [232.]
        ['2020-02-10/17:40:00', '2020-02-10/17:50:00']
    
    For negBy yr4_currdata
    #df_curr = df_currdata.drop([896]) #yr4 negBy drop
    
    For negBy yr2_currdata
    #df_curr = df_currdata.drop([234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244])# yr2 negBy drop
    #df_curr = df_curr.reset_index(drop=True)
    
    For SoBz yr2_currData
    #df_curr2_yr2 = df_currdata_yr2.drop([189, 190, 191, 192, 193, 194, 195]) #dropping NoData row for Yr2
    #df_curr2_yr2 = df_curr2_yr2.reset_index(drop = True) #reset index# column for new DF
    
    For SoBz yr3_currData
    #df_curr2_yr3 = df_currdata_yr3.drop([393, 394]) #dropping NoData row for Yr3
    #df_curr2_yr3 = df_curr2_yr3.reset_index(drop = True) #reset index# column for new DF
    ''';
    return;



#checking_curr_dfs(df_noBzcurrdata_yr1, df_noBz_noslash10m_yr1, drop_yr1, trange_norBztuple_yr1)

#checking_curr_dfs(df_noBzcurrdata_yr2, df_noBz_noslash10m_yr2, drop_yr2, trange_norBztuple_yr2)

#checking_curr_dfs(df_noBzcurrdata_yr3, df_noBz_noslash10m_yr3, drop_yr3, trange_norBztuple_yr3)

#checking_curr_dfs(df_noBzcurrdata_yr4, df_noBz_noslash10m_yr4, drop_yr4, trange_norBztuple_yr4)

#checking_curr_dfs(df_noBzcurrdata_yr5, df_noBz_noslash10m_yr5, drop_yr5, trange_norBztuple_yr5)



* **(11-14-2021)The following comments apply to SoBz Current Data extracted way back when:** 
    * (9-21-2021) As resolved by Rick, the reason why an IndexError occurred for Extractin Yr2 current data on the trange `['2017-01-02/07:35:00', '2017-01-02/07:45:00']`, was that Probe 3 had to recorded data for that period
        * As such, I will implement a `try-except` code to avoid my code from halting when an error occurs and have it record the time intervals in which such errors do occur
            * Same no data error occurred for `Input date was: ['2018-05-27/09:30:00', '2018-05-27/09:40:00']`. Which sucks cause year 3 was almost done. This just re-enforces my need to implement a try-except feature.
    * (9-23-2021) After implementing `try-except` code:
        * Yr 3 Missing data trange Input date: `['2018-05-27/09:30:00', '2018-05-27/09:40:00']` and `['2018-05-27/10:40:00', '2018-05-27/10:50:00'] `, or droplist_ind = [393, 394]
        * Yr 2 Missing data trange inputs, droplist_index =  [189. 190. 191. 192. 193. 194. 195.] or
~~~
        ['2017-01-02/07:35:00', '2017-01-02/07:45:00']
        ['2017-01-02/08:05:00', '2017-01-02/08:15:00']
        ['2017-01-02/10:35:00', '2017-01-02/10:45:00']
        ['2017-01-02/11:55:00', '2017-01-02/12:05:00']
        ['2017-01-02/13:45:00', '2017-01-02/13:55:00']
        ['2017-01-02/14:20:00', '2017-01-02/14:30:00']
        ['2017-01-03/09:50:00', '2017-01-03/10:00:00']
~~~

* **(11-23-2021)** Comments about which tranges to drop for each Clockang Current Dataframe:
~~~
    For norBz yr5_currdata
    #df_curr = df_currdata.drop([131]) #yr5 norBz drop
        DropList index: [131.]
        ['2020-02-10/20:15:00', '2020-02-10/20:25:00']


    For norBz yr3_currdata
    #df_curr = df_currdata([113,114,115,272]) #yr3 norBz drop
        DropList index: [113. 114. 115. 272.]
        ['2018-05-27/04:50:00', '2018-05-27/05:00:00']
        ['2018-05-27/05:20:00', '2018-05-27/05:30:00']
        ['2018-05-27/21:40:00', '2018-05-27/21:50:00']
        ['2018-05-27/22:30:00', '2018-05-27/22:40:00']
    
    
    
    For norBz yr2_currdata
    #df_curr = df_currdata.drop([105]) #yr2 norBz drop
    DropList index: [105.]
    ['2017-01-04/16:40:00', '2017-01-04/16:50:00']
    
    
    For posBy yr5_currdata
    #df_curr = df_currdata.drop([251, 252, 253]) # yr5 posBy drop
        DropList index: [251. 252. 253.]
        ['2020-02-10/21:10:00', '2020-02-10/21:20:00']
        ['2020-02-10/22:55:00', '2020-02-10/23:05:00']
        ['2020-02-10/23:25:00', '2020-02-10/23:35:00']
        
        
    For posBy yr3_currdata
    #df_curr = df_currdata.drop([689, 690, 691, 692, 693, 694, 695, 696])# yr3 posBy drop
        DropList index: [689. 690. 691. 692. 693. 694. 695. 696.]
        ['2018-05-27/14:20:00', '2018-05-27/14:30:00']
        ['2018-05-27/14:50:00', '2018-05-27/15:00:00']
        ['2018-05-27/16:55:00', '2018-05-27/17:05:00']
        ['2018-05-27/17:30:00', '2018-05-27/17:40:00']
        ['2018-05-27/18:00:00', '2018-05-27/18:10:00']
        ['2018-05-27/18:30:00', '2018-05-27/18:40:00']
        ['2018-05-27/19:25:00', '2018-05-27/19:35:00']
        ['2018-05-27/20:10:00', '2018-05-27/20:20:00']



    For posBy yr2_currdata
    #df_curr = df_currdata.drop([481, 482, 483, 484, 485, 486, 487, 488, 489, 490])# yr2 posBy drop
        #DropList index: [481. 482. 483. 484. 485. 486. 487. 488. 489. 490.]
        ['2017-01-02/14:55:00', '2017-01-02/15:05:00']
        ['2017-01-02/15:25:00', '2017-01-02/15:35:00']
        ['2017-01-02/16:50:00', '2017-01-02/17:00:00']
        ['2017-01-02/17:20:00', '2017-01-02/17:30:00']
        ['2017-01-02/23:50:00', '2017-01-03/00:00:00']
        ['2017-01-03/01:20:00', '2017-01-03/01:30:00']
        ['2017-01-03/01:55:00', '2017-01-03/02:05:00']
        ['2017-01-03/02:25:00', '2017-01-03/02:35:00']
        ['2017-01-03/03:05:00', '2017-01-03/03:15:00']
        ['2017-01-03/05:05:00', '2017-01-03/05:15:00']

    For negBy yr5_currdata
    #df_curr = df_currdata.drop([232]) #yr5 negBy drop
        DropList index: [232.]
        ['2020-02-10/17:40:00', '2020-02-10/17:50:00']
    
    For negBy yr4_currdata
    #df_curr = df_currdata.drop([896]) #yr4 negBy drop
    
    For negBy yr2_currdata
    #df_curr = df_currdata.drop([234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244])# yr2 negBy drop
    #df_curr = df_curr.reset_index(drop=True)
    
    For SoBz yr2_currData
    #df_curr2_yr2 = df_currdata_yr2.drop([189, 190, 191, 192, 193, 194, 195]) #dropping NoData row for Yr2
    #df_curr2_yr2 = df_curr2_yr2.reset_index(drop = True) #reset index# column for new DF
    
    For SoBz yr3_currData
    #df_curr2_yr3 = df_currdata_yr3.drop([393, 394]) #dropping NoData row for Yr3
    #df_curr2_yr3 = df_curr2_yr3.reset_index(drop = True) #reset index# column for new DF
~~~


### Section 2.2b: (1-13-2022)
#### Testing/Exporting Newly Created Current DFs and creating Corresponding MEC DF

1. Check if time arrays in currDF match onto inputted time intervals
2. Create corresponding mecDF that matches currDF time intervals
3. Removing currData rows(and corresponding mecData rows) whose elements contain 'NoData' due to the errors that were encountered in the extraction algorithm


In [ ]:
def compare_currmec_start(df_curr, df_mec):
    """Compares dataframes columns for Start-dates between MECposdata and CurrData DFs
    They should be the same"""
    #Extracting and creating copies of DF start-date column arrays
    currSTR_start = df_curr['Cur_start'].to_numpy(copy=True)
    mec_start = df_mec['Start_str'].to_numpy(copy=True)
    print('CurrData and MECpos Dataframes start-date columns are the same: ', (mec_start == currSTR_start).all())
    #print('Using np.array_equals, the start-date columns are the same: ', np.array_equal(currSTR_start, mec_start))
    display(df_curr.iloc[:3])
    display(df_mec[:3])
    #Comparing individual elements to check for specific non-matching elements
    n = 0
    for i in range(len(mec_start)):
        if currSTR_start[i] == mec_start[i]:
            n = n+1
        else:
            print("MEC-ind %d Didn't match: "%(i),currSTR_start[i], 'and', mec_start[i])
    print('With a length of %d, %d start-date elements match between Currdata and MECpos'%(len(mec_start),n))
    return; 

def check_currMEC_dfs(df_currdata, df_noslash10, yrX_mec, droplist, trange_tuple):
    """Creating MECpos DFs whose date columns arrays match the newly created currData DFs.
    Ensure currData start-date column arrays match trange input and lastly compare exported/imported DFs"""
    print('Data being analyzed:\n', namestr(df_currdata, globals()),namestr(df_noslash10, globals()),
         namestr(yrX_mec, globals()), namestr(droplist, globals()) ,namestr(trange_tuple, globals()))
    #Creating mecDF from OG Last10m interval and comparing both start-end date column arrays
    df_MECpos = get_MEC_df(df_noslash10, yrX_mec)
    compare_mec_start_end(df_MECpos, df_noslash10)
    #Display any 'NoData' errors that occured thorughout any tranges for currData extraction
    anDrop = [int(item) for item in droplist]
    #anDrop = int(droplist) #ensure droplist elements are integers
    if len(anDrop) > 0:
        print('DropList index:', anDrop)
        for i in range(len(anDrop)):
            print(trange_tuple[(anDrop[i])])  
            # displays Drop tuple element index with index recorded by drop list
            # int(droplist[i]) ensures input into Date is an integer to avoid error
    else:
        print('No data errors detected for any tranges this year\n')
    #Compare yrX currData and OG Last10m start-date columns
    compare_currstart(df_currdata, df_noslash10)
    #Based on droplist, either omit NoData rows or leave alone the mec and curDFs
    #Create copies of inputted DFs
    df_curcop = df_currdata.copy(deep = True) 
    df_meccop = df_MECpos.copy(deep=True) 
    if len(anDrop) >0: #omit NoData rows from both curr and mec DFs
        #Drop NoData rows
        print('\n Omitting NoData row')
        df_curcop = df_curcop.drop(anDrop)
        df_meccop = df_meccop.drop(anDrop) 
        #Reset index# column for new DF
        df_curcop = df_curcop.reset_index(drop=True) 
        df_meccop = df_meccop.reset_index(drop=True)
        #compare_currstart(df_curcop, df_noslash10) #compared altered currDF to OG Last10m
    # Compare unaltered/altered DFs datecolumns to OG Last10m and with each other
    #Compare date-columns between mec and currDFs
    compare_currmec_start(df_curcop, df_meccop) #they have matching start dates
    
    #Export yrX_MECcurmatch DF and compare to imported version
    #df_meccop.to_csv('pdFile_MECmatch_noGAP_curr2std_L10m_negByled_Yr5.txt', sep = '\t', index = False)
    #df_imp_MECpos = pd.read_csv('pdFile_MECmatch_noGAP_curr2std_L10m_negByled_Yr5.txt', sep = '\t') #mecmatch_noGAP
    #compare_imp_df(df_meccop, df_imp_MECpos)
    
    #Export yrX_currData DF and compare to imported version
    #df_curcop.to_csv('pdFile_CurrPeak_noMECgap_negByled_2stdprom07_Yr5.txt', sep = '\t', index = False)
    #df_imp_currdata = pd.read_csv('pdFile_CurrPeak_noMECgap_negByled_2stdprom07_Yr5.txt', sep = '\t') #currpeak_noMECgap
    #compare_imp_df(df_curcop, df_imp_currdata)

    return;


#check_curr_make_MEC_dfs(df_currdata, df_noslash10, yrX_mec, droplist, trange_tuple)

#check_currMEC_dfs(df_negBycurrdata_yr1, df_nBy_noslash10m_yr1, yr1_MECposdata, drop_yr1, trange_negBytuple_yr1)

#check_currMEC_dfs(df_negBycurrdata_yr2, df_nBy_noslash10m_yr2, yr2_MECposdata, drop_yr2, trange_negBytuple_yr2)

#check_currMEC_dfs(df_negBycurrdata_yr3, df_nBy_noslash10m_yr3, yr3_MECposdata, drop_yr3, trange_negBytuple_yr3)

#check_currMEC_dfs(df_negBycurrdata_yr4, df_nBy_noslash10m_yr4, yr4_MECposdata, drop_yr4, trange_negBytuple_yr4)

#check_currMEC_dfs(df_negBycurrdata_yr5, df_nBy_noslash10m_yr5, yr5_MECposdata, drop_yr5, trange_negBytuple_yr5)

